In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install opencv-python opencv-contrib-python numpy


In [ ]:
import cv2
import matplotlib.pyplot as plt

# Lecture des deux images
input1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png') # image de gauche
input2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png') # image de droite

# Convertir les images en RGB pour l'affichage avec Matplotlib
input1_rgb = cv2.cvtColor(input1, cv2.COLOR_BGR2RGB)
input2_rgb = cv2.cvtColor(input2, cv2.COLOR_BGR2RGB)

# Afficher les images en couleur avec les coordonnées
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.imshow(input1_rgb)
plt.title('Image Gauche')
plt.xlabel('Coordonnées en pixels')
plt.ylabel('Coordonnées en pixels')


plt.subplot(1, 2, 2)
plt.imshow(input2_rgb)
plt.title('Image Droite')
plt.xlabel('Coordonnées en pixels')
plt.ylabel('Coordonnées en pixels')


plt.show()


In [ ]:
import cv2
import matplotlib.pyplot as plt

# Lecture des deux images
input1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png') # image de gauche
input2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png') # image de droite

# Conversion en niveaux de gris
img1 = cv2.cvtColor(input1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(input2, cv2.COLOR_BGR2GRAY)

# Initialiser le détecteur SIFT
sift = cv2.SIFT_create(nfeatures=2000)

# Détecter les points d'intérêt et les descripteurs
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# Dessiner les points d'intérêt sur les images
img1_keypoints = cv2.drawKeypoints(img1, kp1, None, color=(0, 255, 0), flags=0)
img2_keypoints = cv2.drawKeypoints(img2, kp2, None, color=(0, 255, 0), flags=0)

# Afficher les images avec les points d'intérêt
plt.figure(figsize=(15, 10))

plt.subplot(1, 2, 1)
plt.imshow(img1_keypoints, cmap='gray')
plt.title('Points SIFT (Image de gauche)')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img2_keypoints, cmap='gray')
plt.title('Points SIFT (Image de droite)')
plt.axis('off')

plt.show()


In [ ]:
import cv2
import matplotlib.pyplot as plt

# Lire les images
img1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png', cv2.IMREAD_GRAYSCALE)

# Initialiser le détecteur SIFT
sift = cv2.SIFT_create()

# Détecter les points d'intérêt et les descripteurs
keypoints_1 = sift.detect(img1, None)
keypoints_2 = sift.detect(img2, None)

# Dessiner les points d'intérêt sur les images
img1_keypoints = cv2.drawKeypoints(img1, keypoints_1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
img2_keypoints = cv2.drawKeypoints(img2, keypoints_2, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Afficher les images avec les points d'intérêt
plt.figure(figsize=(15, 10))

plt.subplot(1, 2, 1)
plt.imshow(img1_keypoints, cmap='gray')
plt.title('Points d\'intérêt - Image gauche')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img2_keypoints, cmap='gray')
plt.title('Points d\'intérêt - Image droite')
plt.axis('off')

plt.show()


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

###### Lecture des deux images
input1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png') # image de gauche
input2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png') # image de droite

# Conversion en niveaux de gris
img1 = cv2.cvtColor(input1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(input2, cv2.COLOR_BGR2GRAY)

###### Détection des points clefs
sift = cv2.SIFT_create()

# Détection des points SIFT et calcul des descripteurs
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

print('Nb de SIFT : ' + str(len(kp1)) + ' (gauche) ' + str(len(kp2)) + ' (droite)')
imgd = cv2.drawKeypoints(img1, kp1, None, flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)
plt.imshow(imgd), plt.title('%i Points SIFT (Image de gauche)' % len(kp1))
plt.show()

pts1 = []
pts2 = []

# Distance L2 pour descripteur SIFT
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
# Extraction de la liste des 2-plus-proches-voisins
matches = bf.knnMatch(des1, des2, k=2)
# Filtrage des appariements par application du ratio test avec un seuil plus permissif
good = []
bad = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)
        good.append([m])
    else:
        bad.append([m])

mfilt_image_good = np.array([])
mfilt_image_bad = np.array([])
draw_params_good = dict(matchColor=(0, 255, 0),
                        singlePointColor=(255, 0, 0),
                        flags=0)
draw_params_bad = dict(matchColor=(0, 0, 255),
                       singlePointColor=(255, 0, 0),
                       flags=0)

mfilt_image_good = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, **draw_params_good)
mfilt_image_bad = cv2.drawMatchesKnn(img1, kp1, img2, kp2, bad, None, **draw_params_bad)
pts1 = np.float32(pts1)
pts2 = np.float32(pts2)
print('Nb de paires sélectionnées : ' + str(pts1.shape[0]))

plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(mfilt_image_good)
plt.title('Appariement filtré (bons) : %i paires conservées' % len(good))
plt.subplot(122), plt.imshow(mfilt_image_bad)
plt.title('Appariement filtré (mauvais) : %i paires conservées' % len(bad))
plt.show()

##### Définition des fonctions auxiliaires
def drawlines(img1, img2, lines, pts1, pts2):
    ''' img1 - image sur laquelle on dessine les épilines pour les points dans img2
        lines - épilines correspondantes '''
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(cv2.cvtColor(np.asarray([[[np.random.randint(0, 180), 255, 255]]], dtype=np.uint8), cv2.COLOR_HSV2BGR)[0, 0, :].tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 2)
        img1 = cv2.circle(img1, (int(pt1[0]), int(pt1[1])), 5, color, -1)
        img2 = cv2.circle(img2, (int(pt2[0]), int(pt2[1])), 5, color, -1)
    return img1, img2

def drawFundamental(img1, img2, pts1, pts2, F):
    # Trouver les épilines correspondant à certains points de l'image de droite (deuxième image)
    # et dessiner leurs lignes sur l'image de gauche
    indexes = np.random.randint(0, pts1.shape[0], size=(10))
    indexes = range(pts1.shape[0])
    samplePt1 = pts1[indexes, :]
    samplePt2 = pts2[indexes, :]

    lines1 = cv2.computeCorrespondEpilines(samplePt2.reshape(-1, 1, 2), 2, F)
    lines1 = lines1.reshape(-1, 3)
    img5, img6 = drawlines(img1, img2, lines1, samplePt1, samplePt2)

    # Trouver les épilines correspondant à certains points de l'image de gauche (première image)
    # et dessiner leurs lignes sur l'image de droite

    lines2 = cv2.computeCorrespondEpilines(samplePt1.reshape(-1, 1, 2), 1, F)
    lines2 = lines2.reshape(-1, 3)
    img3, img4 = drawlines(img2, img1, lines2, samplePt2, samplePt1)
    return img5, img3

###### Calcul de la Matrice Fondamentale avec OpenCV RANSAC
FRansac, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC,
                                       ransacReprojThreshold=0.99,  # Distance max de reprojection en pixels pour un inlier
                                       confidence=0.99)  # Niveau de confiance désiré
print('Nb inliers RANSAC : ' + str(mask.sum()))

# Affichage des inliers uniquement
inlierpts1 = pts1[mask.ravel() == 1]
inlierpts2 = pts2[mask.ravel() == 1]

# Tracer les droites épipolaires
imgL, imgR = drawFundamental(img1, img2, inlierpts1, inlierpts2, FRansac)
plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(imgL)
plt.title('Lignes épipolaires des %i inliers (gauche)' % mask.sum())
plt.subplot(122), plt.imshow(imgR)
plt.title('Lignes épipolaires des %i inliers (droite)' % mask.sum())
plt.show()


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

###### Lecture des deux images
# input1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/obj_left.png') # image de droite
# input2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/obj_right.png') # image de gauche
input1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png') # image de droite
input2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png') # image de gauche
# Conversion en niveaux de gris
img1 = cv2.cvtColor(input1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(input2, cv2.COLOR_BGR2GRAY)

###### Détection des points clefs
kaze = cv2.KAZE_create()

# Détection des points KAZE et calcul des descripteurs M-SURF
kp1, des1 = kaze.detectAndCompute(img1, None)
kp2, des2 = kaze.detectAndCompute(img2, None)

print('Nb de SIFT : ' + str(len(kp1)) + ' (gauche) ' + str(len(kp2)) + ' (droite)')
imgd = cv2.drawKeypoints(img1, kp1, None, -1, flags=cv2.DrawMatchesFlags_DEFAULT)
plt.imshow(imgd), plt.title('%i Points SIFT (Image de gauche)' % len(kp1))
plt.show()

pts1 = []
pts2 = []

# Distance L2 pour descripteur M-SURF (KAZE)
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
# Extraction de la liste des 2-plus-proches-voisins
matches = bf.knnMatch(des1, des2, k=2)
# Filtrage des appariements par application du ratio test
good = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)
        good.append([m])

mfilt_image = np.array([])
draw_params = dict(matchColor=(0, 255, 0),
                   singlePointColor=(255, 0, 0),
                   flags=0)
mfilt_image = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, **draw_params)
pts1 = np.float32(pts1)
pts2 = np.float32(pts2)
print('Nb de paires sélectionnées : ' + str(pts1.shape[0]))

plt.figure(figsize=(15, 5))
plt.imshow(mfilt_image)
plt.title('Appariement filtré : %i paires conservées' % pts1.shape[0])
plt.show()

##### Définition des fonctions auxiliaires
def drawlines(img1, img2, lines, pts1, pts2):
    ''' img1 - image sur laquelle on dessine les épilines pour les points dans img2
        lines - épilines correspondantes '''
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(cv2.cvtColor(np.asarray([[[np.random.randint(0, 180), 255, 255]]], dtype=np.uint8), cv2.COLOR_HSV2BGR)[0, 0, :].tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 2)
        img1 = cv2.circle(img1, (int(pt1[0]), int(pt1[1])), 5, color, -1)
        img2 = cv2.circle(img2, (int(pt2[0]), int(pt2[1])), 5, color, -1)
    return img1, img2

def drawFundamental(img1, img2, pts1, pts2, F):
    # Trouver les épilines correspondant à certains points de l'image de droite (deuxième image)
    # et dessiner leurs lignes sur l'image de gauche
    indexes = np.random.randint(0, pts1.shape[0], size=(10))
    indexes = range(pts1.shape[0])
    samplePt1 = pts1[indexes, :]
    samplePt2 = pts2[indexes, :]

    lines1 = cv2.computeCorrespondEpilines(samplePt2.reshape(-1, 1, 2), 2, F)
    lines1 = lines1.reshape(-1, 3)
    img5, img6 = drawlines(img1, img2, lines1, samplePt1, samplePt2)

    # Trouver les épilines correspondant à certains points de l'image de gauche (première image)
    # et dessiner leurs lignes sur l'image de droite

    lines2 = cv2.computeCorrespondEpilines(samplePt1.reshape(-1, 1, 2), 1, F)
    lines2 = lines2.reshape(-1, 3)
    img3, img4 = drawlines(img2, img1, lines2, samplePt2, samplePt1)
    return img5, img3

###### Calcul de la Matrice Fondamentale avec OpenCV RANSAC
FRansac, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC,
                                       ransacReprojThreshold=0.99,  # Distance max de reprojection en pixels pour un inlier
                                       confidence=0.99)  # Niveau de confiance désiré
print('Nb inliers RANSAC : ' + str(mask.sum()))

# Affichage des inliers uniquement
inlierpts1 = pts1[mask.ravel() == 1]
inlierpts2 = pts2[mask.ravel() == 1]

# Tracer les droites épipolaires
imgL, imgR = drawFundamental(img1, img2, inlierpts1, inlierpts2, FRansac)
plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(imgL)
plt.title('Lignes épipolaires des %i inliers (gauche)' % mask.sum())
plt.subplot(122), plt.imshow(imgR)
plt.title('Lignes épipolaires des %i inliers (droite)' % mask.sum())
plt.show()


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

###### Lecture des deux images
# input1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/obj_left.png') # image de droite
# input2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/obj_right.png') # image de gauche
input1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png') # image de droite
input2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png') # image de gauche
# Conversion en niveaux de gris
img1 = cv2.cvtColor(input1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(input2, cv2.COLOR_BGR2GRAY)

###### Détection des points clefs
kaze = cv2.KAZE_create()

# Détection des points KAZE et calcul des descripteurs M-SURF
kp1, des1 = kaze.detectAndCompute(img1, None)
kp2, des2 = kaze.detectAndCompute(img2, None)

print('Nb de SIFT : ' + str(len(kp1)) + ' (gauche) ' + str(len(kp2)) + ' (droite)')
imgd = cv2.drawKeypoints(img1, kp1, None, -1, flags=cv2.DrawMatchesFlags_DEFAULT)
plt.imshow(imgd), plt.title('%i Points SIFT (Image de gauche)' % len(kp1))
plt.show()

pts1 = []
pts2 = []

# Distance L2 pour descripteur M-SURF (KAZE)
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
# Extraction de la liste des 2-plus-proches-voisins
matches = bf.knnMatch(des1, des2, k=2)
# Filtrage des appariements par application du ratio test
good = []
for m, n in matches:
    if m.distance < 0.1 * n.distance:
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)
        good.append([m])

mfilt_image = np.array([])
draw_params = dict(matchColor=(0, 255, 0),
                   singlePointColor=(255, 0, 0),
                   flags=0)
mfilt_image = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, **draw_params)
pts1 = np.float32(pts1)
pts2 = np.float32(pts2)
print('Nb de paires sélectionnées : ' + str(pts1.shape[0]))

plt.figure(figsize=(15, 5))
plt.imshow(mfilt_image)
plt.title('Appariement filtré : %i paires conservées' % pts1.shape[0])
plt.show()

##### Définition des fonctions auxiliaires
def drawlines(img1, img2, lines, pts1, pts2):
    ''' img1 - image sur laquelle on dessine les épilines pour les points dans img2
        lines - épilines correspondantes '''
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(cv2.cvtColor(np.asarray([[[np.random.randint(0, 180), 255, 255]]], dtype=np.uint8), cv2.COLOR_HSV2BGR)[0, 0, :].tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 2)
        img1 = cv2.circle(img1, (int(pt1[0]), int(pt1[1])), 5, color, -1)
        img2 = cv2.circle(img2, (int(pt2[0]), int(pt2[1])), 5, color, -1)
    return img1, img2

def drawFundamental(img1, img2, pts1, pts2, F):
    # Trouver les épilines correspondant à certains points de l'image de droite (deuxième image)
    # et dessiner leurs lignes sur l'image de gauche
    indexes = np.random.randint(0, pts1.shape[0], size=(10))
    indexes = range(pts1.shape[0])
    samplePt1 = pts1[indexes, :]
    samplePt2 = pts2[indexes, :]

    lines1 = cv2.computeCorrespondEpilines(samplePt2.reshape(-1, 1, 2), 2, F)
    lines1 = lines1.reshape(-1, 3)
    img5, img6 = drawlines(img1, img2, lines1, samplePt1, samplePt2)

    # Trouver les épilines correspondant à certains points de l'image de gauche (première image)
    # et dessiner leurs lignes sur l'image de droite

    lines2 = cv2.computeCorrespondEpilines(samplePt1.reshape(-1, 1, 2), 1, F)
    lines2 = lines2.reshape(-1, 3)
    img3, img4 = drawlines(img2, img1, lines2, samplePt2, samplePt1)
    return img5, img3

###### Calcul de la Matrice Fondamentale avec OpenCV RANSAC
FRansac, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC,
                                       ransacReprojThreshold=0.99,  # Distance max de reprojection en pixels pour un inlier
                                       confidence=0.99)  # Niveau de confiance désiré
print('Nb inliers RANSAC : ' + str(mask.sum()))

# Affichage des inliers uniquement
inlierpts1 = pts1[mask.ravel() == 1]
inlierpts2 = pts2[mask.ravel() == 1]

# Tracer les droites épipolaires
imgL, imgR = drawFundamental(img1, img2, inlierpts1, inlierpts2, FRansac)
plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(imgL)
plt.title('Lignes épipolaires des %i inliers (gauche)' % mask.sum())
plt.subplot(122), plt.imshow(imgR)
plt.title('Lignes épipolaires des %i inliers (droite)' % mask.sum())
plt.show()


In [ ]:
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

def load_images(left_image_path, right_image_path):
    if not os.path.exists(left_image_path):
        raise FileNotFoundError(f"Left image not found at {left_image_path}")
    if not os.path.exists(right_image_path):
        raise FileNotFoundError(f"Right image not found at {right_image_path}")

    img_left = cv2.imread(left_image_path, cv2.IMREAD_GRAYSCALE)
    img_right = cv2.imread(right_image_path, cv2.IMREAD_GRAYSCALE)

    if img_left is None or img_right is None:
        raise FileNotFoundError("One of the images was not found or could not be loaded.")

    return img_left, img_right

def detect_sift_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if keypoints is None or descriptors is None:
        raise ValueError("No keypoints or descriptors found in the image.")
    return keypoints, descriptors

def match_features(descriptors_left, descriptors_right):
    index_params = dict(algorithm=1, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(descriptors_left, descriptors_right, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
    return good_matches

def draw_matches(img_left, img_right, keypoints_left, keypoints_right, good_matches):
    img_matches = cv2.drawMatches(img_left, keypoints_left, img_right, keypoints_right, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    return img_matches

def compute_disparity_map(img_left, img_right):
    stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)
    disparity = stereo.compute(img_left, img_right)
    return disparity

def main():
    left_image_path = '/content/drive/MyDrive/COMPUTER_VISION/obj_left.png'
    right_image_path = '/content/drive/MyDrive/COMPUTER_VISION/obj_right.png'

    img_left, img_right = load_images(left_image_path, right_image_path)

    keypoints_left, descriptors_left = detect_sift_features(img_left)
    keypoints_right, descriptors_right = detect_sift_features(img_right)

    good_matches = match_features(descriptors_left, descriptors_right)

    img_matches = draw_matches(img_left, img_right, keypoints_left, keypoints_right, good_matches)

    disparity = compute_disparity_map(img_left, img_right)
    disp_norm = cv2.normalize(disparity, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    disp_norm = np.uint8(disp_norm)

    # Affichage des résultats avec matplotlib
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(img_left, cmap='gray')
    plt.title('Image I(x,y)')

    plt.subplot(1, 3, 2)
    plt.imshow(disp_norm, cmap='gray')
    plt.title('Carte des disparités D(x,y)')

    plt.subplot(1, 3, 3)
    plt.imshow(img_right, cmap='gray')
    plt.title("Image I'(x', y')")

    plt.show()

if __name__ == '__main__':
    main()


In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt

def load_images(left_image_path, right_image_path):
    if not os.path.exists(left_image_path):
        raise FileNotFoundError(f"Left image not found at {left_image_path}")
    if not os.path.exists(right_image_path):
        raise FileNotFoundError(f"Right image not found at {right_image_path}")

    img_left = cv2.imread(left_image_path, cv2.IMREAD_GRAYSCALE)
    img_right = cv2.imread(right_image_path, cv2.IMREAD_GRAYSCALE)

    if img_left is None or img_right is None:
        raise FileNotFoundError("One of the images was not found or could not be loaded.")

    return img_left, img_right

def detect_sift_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if keypoints is None or descriptors is None:
        raise ValueError("No keypoints or descriptors found in the image.")
    return keypoints, descriptors

def match_features(descriptors_left, descriptors_right):
    index_params = dict(algorithm=1, trees=5)
    search_params = dict(checks=30)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(descriptors_left, descriptors_right, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.99 * n.distance:
            good_matches.append(m)
    return good_matches

def filter_matches_ransac(good_matches, keypoints_left, keypoints_right):
    if len(good_matches) > 4:
        src_pts = np.float32([keypoints_left[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints_right[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        matches_mask = mask.ravel().tolist()

        ransac_matches = [good_matches[i] for i in range(len(good_matches)) if matches_mask[i]]

        return ransac_matches
    else:
        return []

def draw_matches(img_left, img_right, keypoints_left, keypoints_right, matches, matches_title):
    img_matches = cv2.drawMatches(img_left, keypoints_left, img_right, keypoints_right, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    plt.figure(figsize=(10, 5))
    plt.imshow(img_matches)
    plt.title(matches_title)
    plt.axis('off')
    plt.show()

def main():
    left_image_path = '/content/drive/MyDrive/COMPUTER_VISION/Left_img.PNG'
    right_image_path = '/content/drive/MyDrive/COMPUTER_VISION/Right_img.PNG'  # Utilise les chemins d'accès corrects pour tes images

    img_left, img_right = load_images(left_image_path, right_image_path)

    keypoints_left, descriptors_left = detect_sift_features(img_left)
    keypoints_right, descriptors_right = detect_sift_features(img_right)

    good_matches = match_features(descriptors_left, descriptors_right)

    # Filtrage avec RANSAC
    ransac_matches = filter_matches_ransac(good_matches, keypoints_left, keypoints_right)

    # Affichage des correspondances avant et après RANSAC
    draw_matches(img_left, img_right, keypoints_left, keypoints_right, good_matches, 'Correspondances initiales')
    draw_matches(img_left, img_right, keypoints_left, keypoints_right, ransac_matches, 'Correspondances filtrées par RANSAC')

if __name__ == '__main__':
    main()


In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt

def load_images(left_image_path, right_image_path):
    if not os.path.exists(left_image_path):
        raise FileNotFoundError(f"Left image not found at {left_image_path}")
    if not os.path.exists(right_image_path):
        raise FileNotFoundError(f"Right image not found at {right_image_path}")

    img_left = cv2.imread(left_image_path, cv2.IMREAD_GRAYSCALE)
    img_right = cv2.imread(right_image_path, cv2.IMREAD_GRAYSCALE)

    if img_left is None or img_right is None:
        raise FileNotFoundError("One of the images was not found or could not be loaded.")

    return img_left, img_right

def detect_sift_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if keypoints is None or descriptors is None:
        raise ValueError("No keypoints or descriptors found in the image.")
    return keypoints, descriptors

def match_features(descriptors_left, descriptors_right):
    index_params = dict(algorithm=1, trees=5)
    search_params = dict(checks=30)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(descriptors_left, descriptors_right, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.10 * n.distance:
            good_matches.append(m)
    return good_matches

def filter_matches_ransac(good_matches, keypoints_left, keypoints_right):
    if len(good_matches) > 5:
        src_pts = np.float32([keypoints_left[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints_right[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        matches_mask = mask.ravel().tolist()

        ransac_matches = [good_matches[i] for i in range(len(good_matches)) if matches_mask[i]]

        return ransac_matches, M
    else:
        return [], None

def draw_matches_custom(img_left, img_right, keypoints_left, keypoints_right, matches, matches_title):
    # Create a combined image
    h1, w1 = img_left.shape
    h2, w2 = img_right.shape
    combined_img = np.zeros((h1 + h2, max(w1, w2), 3), dtype='uint8')
    combined_img[:h1, :w1] = cv2.cvtColor(img_left, cv2.COLOR_GRAY2BGR)
    combined_img[h1:h1 + h2, :w2] = cv2.cvtColor(img_right, cv2.COLOR_GRAY2BGR)

    for match in matches:
        pt1 = (int(keypoints_left[match.queryIdx].pt[0]), int(keypoints_left[match.queryIdx].pt[1]))
        pt2 = (int(keypoints_right[match.trainIdx].pt[0]), int(keypoints_right[match.trainIdx].pt[1] + h1))

        cv2.circle(combined_img, pt1, 3, (0, 255, 0), -1)
        cv2.circle(combined_img, pt2, 3, (0, 255, 0), -1)
        cv2.line(combined_img, pt1, pt2, (0, 0, 255), 1)

    plt.imshow(combined_img)
    plt.title(matches_title)
    plt.axis('off')
    plt.show()

def main():
    left_image_path = '/content/drive/MyDrive/COMPUTER_VISION/imLeft.png'
    right_image_path = '/content/drive/MyDrive/COMPUTER_VISION/imRight.png'

    img_left, img_right = load_images(left_image_path, right_image_path)

    keypoints_left, descriptors_left = detect_sift_features(img_left)
    keypoints_right, descriptors_right = detect_sift_features(img_right)

    good_matches = match_features(descriptors_left, descriptors_right)

    # Filtrage avec RANSAC
    ransac_matches, H = filter_matches_ransac(good_matches, keypoints_left, keypoints_right)

    # Transformation de l'image gauche pour ressembler à l'image droite
    height, width = img_right.shape
    img_left_warp = cv2.warpPerspective(img_left, H, (width, height))

    # Affichage des correspondances avant et après RANSAC dans une disposition verticale
    fig, axs = plt.subplots(4, 1, figsize=(10, 20))

    axs[0].imshow(cv2.cvtColor(img_left, cv2.COLOR_GRAY2BGR))
    axs[0].set_title('Image Gauche')
    axs[0].axis('off')

    axs[1].imshow(cv2.cvtColor(img_right, cv2.COLOR_GRAY2BGR))
    axs[1].set_title('Image Droite')
    axs[1].axis('off')

    draw_matches_custom(img_left, img_right, keypoints_left, keypoints_right, good_matches, 'Correspondances Initiales')
    draw_matches_custom(img_left, img_right, keypoints_left, keypoints_right, ransac_matches, 'Correspondances Filtrées par RANSAC')

    plt.show()

    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(img_left_warp, cv2.COLOR_GRAY2BGR))
    plt.title("Transformation de l'image gauche pour ressembler à la droite")
    plt.axis('off')
    plt.show()

if __name__ == '__main__':
    main()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist


# **Chargement et affichage des images**

In [ ]:
# Charger les images stéréoscopiques
# img_left = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/POP01.jpg', cv2.IMREAD_GRAYSCALE)
# img_right = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/POP02.jpg', cv2.IMREAD_GRAYSCALE)
img_left = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png', cv2.IMREAD_GRAYSCALE)
img_right = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png', cv2.IMREAD_GRAYSCALE)

# Affichage des images
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Image Gauche")
plt.imshow(img_left, cmap='gray')
plt.subplot(1, 2, 2)
plt.title("Image Droite")
plt.imshow(img_right, cmap='gray')
plt.show()


# **Détection des points SIFT et correspondance des points**

In [ ]:
# Création du détecteur SIFT
sift = cv2.SIFT_create()

# Détection des points d'intérêt SIFT
keypoints_left, descriptors_left = sift.detectAndCompute(img_left, None)
keypoints_right, descriptors_right = sift.detectAndCompute(img_right, None)

# Correspondance des points d'intérêt avec le matcher BFMatcher
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(descriptors_left, descriptors_right)

# Tri des correspondances par distance
matches = sorted(matches, key=lambda x: x.distance)

# Affichage des 10 meilleures correspondances
img_matches = cv2.drawMatches(img_left, keypoints_left, img_right, keypoints_right, matches[:650], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.figure(figsize=(15, 10))
plt.title("Correspondances SIFT")
plt.imshow(img_matches)
plt.show()


# **Affinement des correspondances avec RANSAC**

In [ ]:
# Extraction des points de correspondance
pts_left = np.float32([keypoints_left[m.queryIdx].pt for m in matches])
pts_right = np.float32([keypoints_right[m.trainIdx].pt for m in matches])

# Utilisation de RANSAC pour affiner les correspondances
H, mask = cv2.findHomography(pts_left, pts_right, cv2.RANSAC, 7.5)
matchesMask = mask.ravel().tolist()

# Affichage des correspondances après RANSAC
img_ransac_matches = cv2.drawMatches(img_left, keypoints_left, img_right, keypoints_right, [m for i, m in enumerate(matches) if matchesMask[i]], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.figure(figsize=(15, 10))
plt.title("Correspondances SIFT après RANSAC")
plt.imshow(img_ransac_matches)
plt.show()


In [ ]:
!pip install opencv-python opencv-contrib-python matplotlib numpy

# **Calcul de la carte de disparité**

In [ ]:

# Importation des bibliothèques nécessaires
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

# Charger les images stéréoscopiques
# img_left = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/shed_left.png', cv2.IMREAD_GRAYSCALE)
# img_right = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/shed_right.png', cv2.IMREAD_GRAYSCALE)
img_left = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png', cv2.IMREAD_GRAYSCALE)
img_right = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png', cv2.IMREAD_GRAYSCALE)

# Assurez-vous que les deux images ont la même taille
height, width = img_left.shape
if img_right.shape != img_left.shape:
    img_right = cv2.resize(img_right, (width, height))

# Affichage des images
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Image Gauche")
plt.imshow(img_left, cmap='gray')
plt.subplot(1, 2, 2)
plt.title("Image Droite")
plt.imshow(img_right, cmap='gray')
plt.show()

# Création du détecteur SIFT
sift = cv2.SIFT_create()

# Détection des points d'intérêt SIFT
keypoints_left, descriptors_left = sift.detectAndCompute(img_left, None)
keypoints_right, descriptors_right = sift.detectAndCompute(img_right, None)

# Correspondance des points d'intérêt avec le matcher BFMatcher
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(descriptors_left, descriptors_right)

# Tri des correspondances par distance
matches = sorted(matches, key=lambda x: x.distance)

# Affichage des 10 meilleures correspondances
img_matches = cv2.drawMatches(img_left, keypoints_left, img_right, keypoints_right, matches[:50], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.figure(figsize=(15, 10))
plt.title("Correspondances SIFT")
plt.imshow(img_matches)
plt.show()

# Extraction des points de correspondance
pts_left = np.float32([keypoints_left[m.queryIdx].pt for m in matches])
pts_right = np.float32([keypoints_right[m.trainIdx].pt for m in matches])

# Utilisation de RANSAC pour affiner les correspondances
H, mask = cv2.findHomography(pts_left, pts_right, cv2.RANSAC, 1.0)
matchesMask = mask.ravel().tolist()

# Affichage des correspondances après RANSAC
img_ransac_matches = cv2.drawMatches(img_left, keypoints_left, img_right, keypoints_right, [m for i, m in enumerate(matches) if matchesMask[i]], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.figure(figsize=(15, 10))
plt.title("Correspondances SIFT après RANSAC")
plt.imshow(img_ransac_matches)
plt.show()

# Calcul de la carte de disparité
stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)
disparity = stereo.compute(img_left, img_right)

# Affichage de la carte de disparité
plt.figure(figsize=(10, 5))
plt.title("Carte de Disparité")
plt.imshow(disparity, 'gray')
plt.show()

# Calcul de la matrice fondamentale
F, mask = cv2.findFundamentalMat(pts_left, pts_right, cv2.FM_RANSAC)

# Sélection des points inliers
pts_left_inliers = pts_left[mask.ravel() == 1]
pts_right_inliers = pts_right[mask.ravel() == 1]

# Calcul des lignes épipolaires pour les points inliers
lines_left = cv2.computeCorrespondEpilines(pts_right_inliers.reshape(-1, 1, 2), 2, F)
lines_left = lines_left.reshape(-1, 3)

lines_right = cv2.computeCorrespondEpilines(pts_left_inliers.reshape(-1, 1, 2), 1, F)
lines_right = lines_right.reshape(-1, 3)

def draw_epilines(img1, img2, lines, pts1, pts2):
    ''' Fonction pour dessiner les lignes épipolaires '''
    r, c = img1.shape
    img1_color = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2_color = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2]/r[1]])
        x1, y1 = map(int, [c, -(r[2]+r[0]*c)/r[1]])
        img1_color = cv2.line(img1_color, (x0, y0), (x1, y1), (0, 255, 0), 1)
        img1_color = cv2.circle(img1_color, (int(pt1[0]), int(pt1[1])), 5, color, -1)
        img2_color = cv2.circle(img2_color, (int(pt2[0]), int(pt2[1])), 5, color, -1)
    return img1_color, img2_color

# Dessiner les lignes épipolaires sur les images
img_left_epilines, img_right_epilines = draw_epilines(img_left, img_right, lines_left, pts_left_inliers, pts_right_inliers)
img_right_epilines, img_left_epilines = draw_epilines(img_right, img_left, lines_right, pts_right_inliers, pts_left_inliers)

# Affichage des images avec les lignes épipolaires
plt.figure(figsize=(15, 10))
plt.subplot(1, 2, 1)
plt.title("Lignes épipolaires sur l'image gauche")
plt.imshow(img_left_epilines)
plt.subplot(1, 2, 2)
plt.title("Lignes épipolaires sur l'image droite")
plt.imshow(img_right_epilines)
plt.show()


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

###### Lecture des deux images
# input1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/obj_left.png') # image de droite
# input2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/obj_right.png') # image de gauche
input1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png') # image de droite
input2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png') # image de gauche
# Conversion en niveaux de gris
img1 = cv2.cvtColor(input1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(input2, cv2.COLOR_BGR2GRAY)

###### Détection des points clefs
kaze = cv2.KAZE_create()

# Détection des points KAZE et calcul des descripteurs M-SURF
kp1, des1 = kaze.detectAndCompute(img1, None)
kp2, des2 = kaze.detectAndCompute(img2, None)

print('Nb de SIFT : ' + str(len(kp1)) + ' (gauche) ' + str(len(kp2)) + ' (droite)')
imgd = cv2.drawKeypoints(img1, kp1, None, -1, flags=cv2.DrawMatchesFlags_DEFAULT)
plt.imshow(imgd), plt.title('%i Points SIFT (Image de gauche)' % len(kp1))
plt.show()

pts1 = []
pts2 = []

# Distance L2 pour descripteur M-SURF (KAZE)
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
# Extraction de la liste des 2-plus-proches-voisins
matches = bf.knnMatch(des1, des2, k=2)
# Filtrage des appariements par application du ratio test
good = []
for m, n in matches:
    if m.distance < 0.1 * n.distance:
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)
        good.append([m])

mfilt_image = np.array([])
draw_params = dict(matchColor=(0, 255, 0),
                   singlePointColor=(255, 0, 0),
                   flags=0)
mfilt_image = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, **draw_params)
pts1 = np.float32(pts1)
pts2 = np.float32(pts2)
print('Nb de paires sélectionnées : ' + str(pts1.shape[0]))

plt.figure(figsize=(15, 5))
plt.imshow(mfilt_image)
plt.title('Appariement filtré : %i paires conservées' % pts1.shape[0])
plt.show()

##### Définition des fonctions auxiliaires
def drawlines(img1, img2, lines, pts1, pts2):
    ''' img1 - image sur laquelle on dessine les épilines pour les points dans img2
        lines - épilines correspondantes '''
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(cv2.cvtColor(np.asarray([[[np.random.randint(0, 180), 255, 255]]], dtype=np.uint8), cv2.COLOR_HSV2BGR)[0, 0, :].tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 2)
        img1 = cv2.circle(img1, (int(pt1[0]), int(pt1[1])), 5, color, -1)
        img2 = cv2.circle(img2, (int(pt2[0]), int(pt2[1])), 5, color, -1)
    return img1, img2

def drawFundamental(img1, img2, pts1, pts2, F):
    # Trouver les épilines correspondant à certains points de l'image de droite (deuxième image)
    # et dessiner leurs lignes sur l'image de gauche
    indexes = np.random.randint(0, pts1.shape[0], size=(10))
    indexes = range(pts1.shape[0])
    samplePt1 = pts1[indexes, :]
    samplePt2 = pts2[indexes, :]

    lines1 = cv2.computeCorrespondEpilines(samplePt2.reshape(-1, 1, 2), 2, F)
    lines1 = lines1.reshape(-1, 3)
    img5, img6 = drawlines(img1, img2, lines1, samplePt1, samplePt2)

    # Trouver les épilines correspondant à certains points de l'image de gauche (première image)
    # et dessiner leurs lignes sur l'image de droite

    lines2 = cv2.computeCorrespondEpilines(samplePt1.reshape(-1, 1, 2), 1, F)
    lines2 = lines2.reshape(-1, 3)
    img3, img4 = drawlines(img2, img1, lines2, samplePt2, samplePt1)
    return img5, img3

###### Calcul de la Matrice Fondamentale avec OpenCV RANSAC
FRansac, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC,
                                       ransacReprojThreshold=0.99,  # Distance max de reprojection en pixels pour un inlier
                                       confidence=0.99)  # Niveau de confiance désiré
print('Nb inliers RANSAC : ' + str(mask.sum()))

# Affichage des inliers uniquement
inlierpts1 = pts1[mask.ravel() == 1]
inlierpts2 = pts2[mask.ravel() == 1]

# Tracer les droites épipolaires
imgL, imgR = drawFundamental(img1, img2, inlierpts1, inlierpts2, FRansac)
plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(imgL)
plt.title('Lignes épipolaires des %i inliers (gauche)' % mask.sum())
plt.subplot(122), plt.imshow(imgR)
plt.title('Lignes épipolaires des %i inliers (droite)' % mask.sum())
plt.show()


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

###### Lecture des deux images
input1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/obj_left.png') # image de droite
input2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/obj_right.png') # image de gauche

# Conversion en niveaux de gris
img1 = cv2.cvtColor(input1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(input2, cv2.COLOR_BGR2GRAY)

###### Détection des points clefs
sift = cv2.SIFT_create()

# Détection des points SIFT et calcul des descripteurs
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

print('Nb de points SIFT : ' + str(len(kp1)) + ' (gauche) ' + str(len(kp2)) + ' (droite)')
imgd = cv2.drawKeypoints(img1, kp1, None, -1, flags=cv2.DrawMatchesFlags_DEFAULT)
plt.imshow(imgd), plt.title('%i Points SIFT (Image de gauche)' % len(kp1))
plt.show()

pts1 = []
pts2 = []

# Distance L2 pour descripteur SIFT
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
# Extraction de la liste des 2-plus-proches-voisins
matches = bf.knnMatch(des1, des2, k=2)
# Filtrage des appariements par application du ratio test
good = []
for m, n in matches:
    if m.distance < 0.9 * n.distance:
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)
        good.append([m])

mfilt_image = np.array([])
draw_params = dict(matchColor=(0, 255, 0),
                   singlePointColor=(255, 0, 0),
                   flags=0)
mfilt_image = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, **draw_params)
pts1 = np.float32(pts1)
pts2 = np.float32(pts2)
print('Nb de paires sélectionnées : ' + str(pts1.shape[0]))

plt.figure(figsize=(15, 5))
plt.imshow(mfilt_image)
plt.title('Appariement filtré : %i paires conservées' % pts1.shape[0])
plt.show()

##### Définition des fonctions auxiliaires
def drawlines(img1, img2, lines, pts1, pts2):
    ''' img1 - image sur laquelle on dessine les épilines pour les points dans img2
        lines - épilines correspondantes '''
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(cv2.cvtColor(np.asarray([[[np.random.randint(0, 180), 255, 255]]], dtype=np.uint8), cv2.COLOR_HSV2BGR)[0, 0, :].tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 2)
        img1 = cv2.circle(img1, (int(pt1[0]), int(pt1[1])), 5, color, -1)
        img2 = cv2.circle(img2, (int(pt2[0]), int(pt2[1])), 5, color, -1)
    return img1, img2

def drawFundamental(img1, img2, pts1, pts2, F):
    # Trouver les épilines correspondant à certains points de l'image de droite (deuxième image)
    # et dessiner leurs lignes sur l'image de gauche
    indexes = np.random.randint(0, pts1.shape[0], size=(10))
    indexes = range(pts1.shape[0])
    samplePt1 = pts1[indexes, :]
    samplePt2 = pts2[indexes, :]

    lines1 = cv2.computeCorrespondEpilines(samplePt2.reshape(-1, 1, 2), 2, F)
    lines1 = lines1.reshape(-1, 3)
    img5, img6 = drawlines(img1, img2, lines1, samplePt1, samplePt2)

    # Trouver les épilines correspondant à certains points de l'image de gauche (première image)
    # et dessiner leurs lignes sur l'image de droite
    lines2 = cv2.computeCorrespondEpilines(samplePt1.reshape(-1, 1, 2), 1, F)
    lines2 = lines2.reshape(-1, 3)
    img3, img4 = drawlines(img2, img1, lines2, samplePt2, samplePt1)
    return img5, img3

###### Calcul de la Matrice Fondamentale avec OpenCV RANSAC
FRansac, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC,
                                       ransacReprojThreshold=0.5,  # Distance max de reprojection en pixels pour un inlier
                                       confidence=0.99)  # Niveau de confiance désiré
print('Nb inliers RANSAC : ' + str(mask.sum()))

# Affichage des inliers uniquement
inlierpts1 = pts1[mask.ravel() == 1]
inlierpts2 = pts2[mask.ravel() == 1]

# Tracer les droites épipolaires
imgL, imgR = drawFundamental(img1, img2, inlierpts1, inlierpts2, FRansac)
plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(imgL)
plt.title('Lignes épipolaires des %i inliers (gauche)' % mask.sum())
plt.subplot(122), plt.imshow(imgR)
plt.title('Lignes épipolaires des %i inliers (droite)' % mask.sum())
plt.show()

###### Calcul de la carte de disparité
# Assurez-vous que les deux images ont la même taille
height, width = img1.shape
if img2.shape != img1.shape:
    img2 = cv2.resize(img2, (width, height))

# Calcul de la carte de disparité avec StereoSGBM
min_disp = 0
num_disp = 16*5  # Doit être multiple de 16
block_size = 5

stereo = cv2.StereoSGBM_create(minDisparity=min_disp,
    numDisparities=num_disp,
    blockSize=block_size,
    P1=8 * 3 * block_size ** 2,
    P2=32 * 3 * block_size ** 2,
    disp12MaxDiff=1,
    uniquenessRatio=10,
    speckleWindowSize=100,
    speckleRange=32,
    preFilterCap=63,
    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY)

disparity = stereo.compute(img1, img2).astype(np.float32) / 16.0

# Affichage de la carte de disparité
plt.figure(figsize=(10, 5))
plt.title("Carte de Disparité")
plt.imshow(disparity, 'gray')
plt.colorbar()
plt.show()


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

###### Lecture des deux images
input1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/obj_left.png') # image de droite
input2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/obj_right.png') # image de gauche

# Conversion en niveaux de gris
img1 = cv2.cvtColor(input1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(input2, cv2.COLOR_BGR2GRAY)

# Pré-traitement des images
img1 = cv2.GaussianBlur(img1, (5, 5), 0)
img2 = cv2.GaussianBlur(img2, (5, 5), 0)

###### Détection des points clefs
sift = cv2.SIFT_create()

# Détection des points SIFT et calcul des descripteurs
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

print('Nb de points SIFT : ' + str(len(kp1)) + ' (gauche) ' + str(len(kp2)) + ' (droite)')
imgd = cv2.drawKeypoints(img1, kp1, None, -1, flags=cv2.DrawMatchesFlags_DEFAULT)
plt.imshow(imgd), plt.title('%i Points SIFT (Image de gauche)' % len(kp1))
plt.show()

pts1 = []
pts2 = []

# Distance L2 pour descripteur SIFT
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
# Extraction de la liste des 2-plus-proches-voisins
matches = bf.knnMatch(des1, des2, k=2)
# Filtrage des appariements par application du ratio test
good = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)
        good.append([m])

mfilt_image = np.array([])
draw_params = dict(matchColor=(0, 255, 0),
                   singlePointColor=(255, 0, 0),
                   flags=0)
mfilt_image = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, **draw_params)
pts1 = np.float32(pts1)
pts2 = np.float32(pts2)
print('Nb de paires sélectionnées : ' + str(pts1.shape[0]))

plt.figure(figsize=(15, 5))
plt.imshow(mfilt_image)
plt.title('Appariement filtré : %i paires conservées' % pts1.shape[0])
plt.show()

##### Définition des fonctions auxiliaires
def drawlines(img1, img2, lines, pts1, pts2):
    ''' img1 - image sur laquelle on dessine les épilines pour les points dans img2
        lines - épilines correspondantes '''
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(cv2.cvtColor(np.asarray([[[np.random.randint(0, 180), 255, 255]]], dtype=np.uint8), cv2.COLOR_HSV2BGR)[0, 0, :].tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 2)
        img1 = cv2.circle(img1, (int(pt1[0]), int(pt1[1])), 5, color, -1)
        img2 = cv2.circle(img2, (int(pt2[0]), int(pt2[1])), 5, color, -1)
    return img1, img2

def drawFundamental(img1, img2, pts1, pts2, F):
    # Trouver les épilines correspondant à certains points de l'image de droite (deuxième image)
    # et dessiner leurs lignes sur l'image de gauche
    indexes = np.random.randint(0, pts1.shape[0], size=(10))
    indexes = range(pts1.shape[0])
    samplePt1 = pts1[indexes, :]
    samplePt2 = pts2[indexes, :]

    lines1 = cv2.computeCorrespondEpilines(samplePt2.reshape(-1, 1, 2), 2, F)
    lines1 = lines1.reshape(-1, 3)
    img5, img6 = drawlines(img1, img2, lines1, samplePt1, samplePt2)

    # Trouver les épilines correspondant à certains points de l'image de gauche (première image)
    # et dessiner leurs lignes sur l'image de droite
    lines2 = cv2.computeCorrespondEpilines(samplePt1.reshape(-1, 1, 2), 1, F)
    lines2 = lines2.reshape(-1, 3)
    img3, img4 = drawlines(img2, img1, lines2, samplePt2, samplePt1)
    return img5, img3

###### Calcul de la Matrice Fondamentale avec OpenCV RANSAC
FRansac, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC,
                                       ransacReprojThreshold=0.5,  # Distance max de reprojection en pixels pour un inlier
                                       confidence=0.99)  # Niveau de confiance désiré
print('Nb inliers RANSAC : ' + str(mask.sum()))

# Affichage des inliers uniquement
inlierpts1 = pts1[mask.ravel() == 1]
inlierpts2 = pts2[mask.ravel() == 1]

# Tracer les droites épipolaires
imgL, imgR = drawFundamental(img1, img2, inlierpts1, inlierpts2, FRansac)
plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(imgL)
plt.title('Lignes épipolaires des %i inliers (gauche)' % mask.sum())
plt.subplot(122), plt.imshow(imgR)
plt.title('Lignes épipolaires des %i inliers (droite)' % mask.sum())
plt.show()

###### Calcul de la carte de disparité
# Assurez-vous que les deux images ont la même taille
height, width = img1.shape
if img2.shape != img1.shape:
    img2 = cv2.resize(img2, (width, height))

# Calcul de la carte de disparité avec StereoSGBM
min_disp = 0
num_disp = 16*11  # Doit être multiple de 16
block_size = 11

stereo = cv2.StereoSGBM_create(minDisparity=min_disp,
    numDisparities=num_disp,
    blockSize=block_size,
    P1=32 * 3 * block_size ** 2,
    P2=64 * 3 * block_size ** 2,
    disp12MaxDiff=1,
    uniquenessRatio=5,
    speckleWindowSize=50,
    speckleRange=32,
    preFilterCap=63,
    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY)

disparity = stereo.compute(img1, img2).astype(np.float32) / 16.0

# Normaliser la disparité pour l'afficher correctement
disparity = (disparity - min_disp) / num_disp

# Affichage de la carte de disparité
plt.figure(figsize=(10, 5))
plt.title("Carte de Disparité")
plt.imshow(disparity, 'gray')
plt.colorbar()
plt.show()

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Lecture des deux images
input1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png')  # image de gauche
input2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png')  # image de droite

# Conversion en niveaux de gris
img1 = cv2.cvtColor(input1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(input2, cv2.COLOR_BGR2GRAY)

# Pré-traitement des images
img1 = cv2.GaussianBlur(img1, (5, 5), 0)
img2 = cv2.GaussianBlur(img2, (5, 5), 0)

# Détection des points clefs
sift = cv2.SIFT_create()

# Détection des points SIFT et calcul des descripteurs
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

print('Nb de points SIFT : ' + str(len(kp1)) + ' (gauche) ' + str(len(kp2)) + ' (droite)')
imgd = cv2.drawKeypoints(img1, kp1, None, -1, flags=cv2.DrawMatchesFlags_DEFAULT)
plt.imshow(imgd), plt.title('%i Points SIFT (Image de gauche)' % len(kp1))
plt.show()

pts1 = []
pts2 = []

# Distance L2 pour descripteur SIFT
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
# Extraction de la liste des 2-plus-proches-voisins
matches = bf.knnMatch(des1, des2, k=2)
# Filtrage des appariements par application du ratio test
good = []
for m, n in matches:
    if m.distance < 0.2 * n.distance:
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)
        good.append([m])

# Limiter à 50 paires de points
pts1 = np.array(pts1[:50])
pts2 = np.array(pts2[:50])
good = good[:50]

# Dessiner les correspondances
draw_params = dict(matchColor=(0, 255, 0),
                   singlePointColor=(255, 0, 0),
                   flags=0)
mfilt_image = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, **draw_params)
print('Nb de paires sélectionnées : ' + str(pts1.shape[0]))

plt.figure(figsize=(15, 5))
plt.imshow(mfilt_image)
plt.title('Appariement filtré : %i paires conservées' % pts1.shape[0])
plt.show()

# Définition des fonctions auxiliaires
def drawlines(img1, img2, lines, pts1, pts2):
    ''' img1 - image sur laquelle on dessine les épilines pour les points dans img2
        lines - épilines correspondantes '''
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(cv2.cvtColor(np.asarray([[[np.random.randint(0, 180), 255, 255]]], dtype=np.uint8), cv2.COLOR_HSV2BGR)[0, 0, :].tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 2)
        img1 = cv2.circle(img1, (int(pt1[0]), int(pt1[1])), 5, color, -1)
        img2 = cv2.circle(img2, (int(pt2[0]), int(pt2[1])), 5, color, -1)
    return img1, img2

def drawFundamental(img1, img2, pts1, pts2, F):
    # Trouver les épilines correspondant à certains points de l'image de droite (deuxième image)
    # et dessiner leurs lignes sur l'image de gauche
    indexes = np.random.randint(0, pts1.shape[0], size=(10))
    samplePt1 = pts1[indexes, :]
    samplePt2 = pts2[indexes, :]

    lines1 = cv2.computeCorrespondEpilines(samplePt2.reshape(-1, 1, 2), 2, F)
    lines1 = lines1.reshape(-1, 3)
    img5, img6 = drawlines(img1, img2, lines1, samplePt1, samplePt2)

    # Trouver les épilines correspondant à certains points de l'image de gauche (première image)
    # et dessiner leurs lignes sur l'image de droite
    lines2 = cv2.computeCorrespondEpilines(samplePt1.reshape(-1, 1, 2), 1, F)
    lines2 = lines2.reshape(-1, 3)
    img3, img4 = drawlines(img2, img1, lines2, samplePt2, samplePt1)
    return img5, img3

# Calcul de la Matrice Fondamentale avec OpenCV RANSAC
FRansac, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC,
                                       ransacReprojThreshold=1.0,  # Distance max de reprojection en pixels pour un inlier
                                       confidence=0.99)  # Niveau de confiance désiré
print('Nb inliers RANSAC : ' + str(mask.sum()))

# Affichage des inliers uniquement
inlierpts1 = pts1[mask.ravel() == 1]
inlierpts2 = pts2[mask.ravel() == 1]

# Tracer les droites épipolaires
imgL, imgR = drawFundamental(img1, img2, inlierpts1, inlierpts2, FRansac)
plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(imgL)
plt.title('Lignes épipolaires des %i inliers (gauche)' % mask.sum())
plt.subplot(122), plt.imshow(imgR)
plt.title('Lignes épipolaires des %i inliers (droite)' % mask.sum())
plt.show()

# Calcul de la carte de disparité
# Assurez-vous que les deux images ont la même taille
height, width = img1.shape
if img2.shape != img1.shape:
    img2 = cv2.resize(img2, (width, height))

# Calcul de la carte de disparité avec StereoSGBM
min_disp = 0
num_disp = 16*3  # Doit être multiple de 16
block_size = 5

stereo = cv2.StereoSGBM_create(minDisparity=min_disp,
    numDisparities=num_disp,
    blockSize=block_size,
    P1=8 * 3 * block_size ** 2,
    P2=32 * 3 * block_size ** 2,
    disp12MaxDiff=1,
    uniquenessRatio=10,
    speckleWindowSize=100,
    speckleRange=32,
    preFilterCap=63,
    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY)

disparity = stereo.compute(img1, img2).astype(np.float32) / 16.0

# Normaliser la disparité pour l'afficher correctement
disparity = (disparity - min_disp) / num_disp

# Appliquer une colormap pour mieux visualiser la disparité
disparity_colormap = cv2.applyColorMap((disparity * 255).astype(np.uint8), cv2.COLORMAP_JET)

# Affichage de la carte de disparité
plt.figure(figsize=(10, 5))
plt.title("Carte de Disparité")
plt.imshow(disparity_colormap)
plt.colorbar()
plt.show()


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Lecture des deux images
input1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png') # image de gauche
input2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png') # image de droite
# Conversion en niveaux de gris
img1 = cv2.cvtColor(input1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(input2, cv2.COLOR_BGR2GRAY)

# Pré-traitement des images
img1 = cv2.GaussianBlur(img1, (5, 5), 0)
img2 = cv2.GaussianBlur(img2, (5, 5), 0)

###### Détection des points clefs
sift = cv2.SIFT_create()

# Détection des points SIFT et calcul des descripteurs
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

print('Nb de points SIFT : ' + str(len(kp1)) + ' (gauche) ' + str(len(kp2)) + ' (droite)')
imgd = cv2.drawKeypoints(img1, kp1, None, -1, flags=cv2.DrawMatchesFlags_DEFAULT)
plt.imshow(imgd), plt.title('%i Points SIFT (Image de gauche)' % len(kp1))
plt.show()

pts1 = []
pts2 = []

# Distance L2 pour descripteur SIFT
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
# Extraction de la liste des 2-plus-proches-voisins
matches = bf.knnMatch(des1, des2, k=2)
# Filtrage des appariements par application du ratio test
good = []
for m, n in matches:
    if m.distance < 0.2 * n.distance:
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)
        good.append([m])

mfilt_image = np.array([])
draw_params = dict(matchColor=(0, 255, 0),
                   singlePointColor=(255, 0, 0),
                   flags=0)
mfilt_image = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, **draw_params)
pts1 = np.float32(pts1)
pts2 = np.float32(pts2)
print('Nb de paires sélectionnées : ' + str(pts1.shape[0]))

plt.figure(figsize=(15, 5))
plt.imshow(mfilt_image)
plt.title('Appariement filtré : %i paires conservées' % pts1.shape[0])
plt.show()

##### Définition des fonctions auxiliaires
def drawlines(img1, img2, lines, pts1, pts2):
    ''' img1 - image sur laquelle on dessine les épilines pour les points dans img2
        lines - épilines correspondantes '''
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(cv2.cvtColor(np.asarray([[[np.random.randint(0, 180), 255, 255]]], dtype=np.uint8), cv2.COLOR_HSV2BGR)[0, 0, :].tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 2)
        img1 = cv2.circle(img1, (int(pt1[0]), int(pt1[1])), 5, color, -1)
        img2 = cv2.circle(img2, (int(pt2[0]), int(pt2[1])), 5, color, -1)
    return img1, img2

def drawFundamental(img1, img2, pts1, pts2, F):
    # Limiter le nombre de points à 100 pour un affichage clair
    if len(pts1) > 100:
        indices = np.random.choice(len(pts1), 100, replace=False)
        pts1 = pts1[indices]
        pts2 = pts2[indices]

    # Trouver les épilines correspondant à certains points de l'image de droite (deuxième image)
    lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2, F)
    lines1 = lines1.reshape(-1, 3)
    img5, img6 = drawlines(img1, img2, lines1, pts1, pts2)

    # Trouver les épilines correspondant à certains points de l'image de gauche (première image)
    lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1, 1, 2), 1, F)
    lines2 = lines2.reshape(-1, 3)
    img3, img4 = drawlines(img2, img1, lines2, pts2, pts1)
    return img5, img3

###### Calcul de la Matrice Fondamentale avec OpenCV RANSAC
FRansac, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC,
                                       ransacReprojThreshold=0.5,  # Distance max de reprojection en pixels pour un inlier
                                       confidence=0.99)  # Niveau de confiance désiré

if mask is not None:
    print('Nb inliers RANSAC : ' + str(mask.sum()))

    # Affichage des inliers uniquement
    inlierpts1 = pts1[mask.ravel() == 1]
    inlierpts2 = pts2[mask.ravel() == 1]

    # Tracer les droites épipolaires
    imgL, imgR = drawFundamental(img1, img2, inlierpts1, inlierpts2, FRansac)
    combined_img = np.hstack((imgL, imgR))
    plt.figure(figsize=(15, 5))
    plt.imshow(combined_img)
    plt.title('Lignes épipolaires des %i inliers' % mask.sum())
    plt.show()

    # Rectification des images
    h1, w1 = img1.shape
    h2, w2 = img2.shape
    _, H1, H2 = cv2.stereoRectifyUncalibrated(np.float32(inlierpts1), np.float32(inlierpts2), FRansac, (w1, h1))

    img1_rectified = cv2.warpPerspective(img1, H1, (w1, h1))
    img2_rectified = cv2.warpPerspective(img2, H2, (w2, h2))

    # Tracer les droites épipolaires sur les images rectifiées
    imgL_rect, imgR_rect = drawFundamental(img1_rectified, img2_rectified, inlierpts1, inlierpts2, FRansac)
    combined_img_rect = np.hstack((imgL_rect, imgR_rect))
    plt.figure(figsize=(15, 5))
    plt.imshow(combined_img_rect)
    plt.title('Images rectifiées avec épilines corrigées (gauche et droite)')
    plt.show()

    # Calcul de la carte de disparité
    # Assurez-vous que les deux images rectifiées ont la même taille
    height, width = img1_rectified.shape
    if img2_rectified.shape != img1_rectified.shape:
        img2_rectified = cv2.resize(img2_rectified, (width, height))

    # Calcul de la carte de disparité avec StereoSGBM
    min_disp = 0
    num_disp = 16*5  # Doit être multiple de 16
    block_size = 5

    stereo = cv2.StereoSGBM_create(minDisparity=min_disp,
        numDisparities=num_disp,
        blockSize=block_size,
        P1=8 * 3 * block_size ** 2,
        P2=32 * 3 * block_size ** 2,
        disp12MaxDiff=1,
        uniquenessRatio=10,
        speckleWindowSize=100,
        speckleRange=32,
        preFilterCap=63,
        mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY)

    disparity = stereo.compute(img1_rectified, img2_rectified).astype(np.float32) / 16.0

    # Normaliser la disparité pour l'afficher correctement
    # Normaliser la disparité pour l'afficher correctement
disparity = (disparity - min_disp) / num_disp

# Appliquer une colormap pour mieux visualiser la disparité
disparity_colormap = cv2.applyColorMap((disparity * 255).astype(np.uint8), cv2.COLORMAP_JET)

# Affichage de la carte de disparité
plt.figure(figsize=(10, 5))
plt.title("Carte de Disparité")
plt.imshow(disparity_colormap)
plt.colorbar()
plt.show()

# Fonction pour dessiner les lignes épipolaires avec correspondance de couleur correcte
def draw_corrected_epilines(img1, img2, pts1, pts2, F):
    # Limiter le nombre de points à 100 pour un affichage clair
    if len(pts1) > 100:
        indices = np.random.choice(len(pts1), 100, replace=False)
        pts1 = pts1[indices]
        pts2 = pts2[indices]

    # Trouver les épilines correspondant à certains points de l'image de droite (deuxième image)
    lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2, F)
    lines1 = lines1.reshape(-1, 3)
    img1_lines, img2_points = drawlines(img1, img2, lines1, pts1, pts2)

    # Trouver les épilines correspondant à certains points de l'image de gauche (première image)
    lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1, 1, 2), 1, F)
    lines2 = lines2.reshape(-1, 3)
    img2_lines, img1_points = drawlines(img2, img1, lines2, pts2, pts1)

    return img1_lines, img2_lines

# Tracer les droites épipolaires avec correspondance de couleur correcte
imgL_corr, imgR_corr = draw_corrected_epilines(img1, img2, inlierpts1, inlierpts2, FRansac)

# Affichage des images avec épilines corrigées
combined_img_corr = np.hstack((imgL_corr, imgR_corr))
plt.figure(figsize=(15, 5))
plt.imshow(combined_img_corr)
plt.title('Lignes épipolaires corrigées (gauche et droite)')
plt.show()

# Rectification des images avec épilines corrigées
img1_rectified_corr = cv2.warpPerspective(img1, H1, (w1, h1))
img2_rectified_corr = cv2.warpPerspective(img2, H2, (w2, h2))

# Tracer les droites épipolaires sur les images rectifiées avec correspondance de couleur correcte
imgL_rect_corr, imgR_rect_corr = draw_corrected_epilines(img1_rectified_corr, img2_rectified_corr, inlierpts1, inlierpts2, FRansac)

# Affichage des images rectifiées avec épilines corrigées
combined_img_rect_corr = np.hstack((imgL_rect_corr, imgR_rect_corr))
plt.figure(figsize=(15, 5))
plt.imshow(combined_img_rect_corr)
plt.title('Images rectifiées avec épilines corrigées (gauche et droite)')
plt.show()


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Lecture des deux images
input1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png') # image de gauche
input2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png') # image de droite
# Conversion en niveaux de gris
img1 = cv2.cvtColor(input1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(input2, cv2.COLOR_BGR2GRAY)

# Pré-traitement des images
img1 = cv2.GaussianBlur(img1, (5, 5), 0)
img2 = cv2.GaussianBlur(img2, (5, 5), 0)

###### Détection des points clefs
sift = cv2.SIFT_create()

# Détection des points SIFT et calcul des descripteurs
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

print('Nb de points SIFT : ' + str(len(kp1)) + ' (gauche) ' + str(len(kp2)) + ' (droite)')
imgd = cv2.drawKeypoints(img1, kp1, None, -1, flags=cv2.DrawMatchesFlags_DEFAULT)
plt.imshow(imgd), plt.title('%i Points SIFT (Image de gauche)' % len(kp1))
plt.show()

pts1 = []
pts2 = []

# Distance L2 pour descripteur SIFT
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
# Extraction de la liste des 2-plus-proches-voisins
matches = bf.knnMatch(des1, des2, k=2)
# Filtrage des appariements par application du ratio test
good = []
for m, n in matches:
    if m.distance < 0.1 * n.distance:
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)
        good.append([m])

mfilt_image = np.array([])
draw_params = dict(matchColor=(0, 255, 0),
                   singlePointColor=(255, 0, 0),
                   flags=0)
mfilt_image = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, **draw_params)
pts1 = np.float32(pts1)
pts2 = np.float32(pts2)
print('Nb de paires sélectionnées : ' + str(pts1.shape[0]))

plt.figure(figsize=(15, 5))
plt.imshow(mfilt_image)
plt.title('Appariement filtré : %i paires conservées' % pts1.shape[0])
plt.show()

##### Définition des fonctions auxiliaires
def drawlines(img1, img2, lines, pts1, pts2):
    ''' img1 - image sur laquelle on dessine les épilines pour les points dans img2
        lines - épilines correspondantes '''
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(cv2.cvtColor(np.asarray([[[np.random.randint(0, 180), 255, 255]]], dtype=np.uint8), cv2.COLOR_HSV2BGR)[0, 0, :].tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 2)
        img1 = cv2.circle(img1, (int(pt1[0]), int(pt1[1])), 5, color, -1)
        img2 = cv2.circle(img2, (int(pt2[0]), int(pt2[1])), 5, color, -1)
    return img1, img2

def drawFundamental(img1, img2, pts1, pts2, F):
    # Trouver les épilines correspondant à certains points de l'image de droite (deuxième image)
    # et dessiner leurs lignes sur l'image de gauche
    indexes = np.random.randint(0, pts1.shape[0], size=(10))
    indexes = range(pts1.shape[0])
    samplePt1 = pts1[indexes, :]
    samplePt2 = pts2[indexes, :]

    lines1 = cv2.computeCorrespondEpilines(samplePt2.reshape(-1, 1, 2), 2, F)
    lines1 = lines1.reshape(-1, 3)
    img5, img6 = drawlines(img1, img2, lines1, samplePt1, samplePt2)

    # Trouver les épilines correspondant à certains points de l'image de gauche (première image)
    # et dessiner leurs lignes sur l'image de droite
    lines2 = cv2.computeCorrespondEpilines(samplePt1.reshape(-1, 1, 2), 1, F)
    lines2 = lines2.reshape(-1, 3)
    img3, img4 = drawlines(img2, img1, lines2, samplePt2, samplePt1)
    return img5, img3

###### Calcul de la Matrice Fondamentale avec OpenCV RANSAC
FRansac, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC,
                                       ransacReprojThreshold=0.5,  # Distance max de reprojection en pixels pour un inlier
                                       confidence=0.99)  # Niveau de confiance désiré
print('Nb inliers RANSAC : ' + str(mask.sum()))

# Affichage des inliers uniquement
inlierpts1 = pts1[mask.ravel() == 1]
inlierpts2 = pts2[mask.ravel() == 1]

# Tracer les droites épipolaires
imgL, imgR = drawFundamental(img1, img2, inlierpts1, inlierpts2, FRansac)
plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(imgL)
plt.title('Lignes épipolaires des %i inliers (gauche)' % mask.sum())
plt.subplot(122), plt.imshow(imgR)
plt.title('Lignes épipolaires des %i inliers (droite)' % mask.sum())
plt.show()

# Rectification des images
h1, w1 = img1.shape
h2, w2 = img2.shape
_, H1, H2 = cv2.stereoRectifyUncalibrated(np.float32(inlierpts1), np.float32(inlierpts2), FRansac, (w1, h1))

img1_rectified = cv2.warpPerspective(img1, H1, (w1, h1))
img2_rectified = cv2.warpPerspective(img2, H2, (w2, h2))

# Tracer les droites épipolaires sur les images rectifiées
imgL_rect, imgR_rect = drawFundamental(img1_rectified, img2_rectified, inlierpts1, inlierpts2, FRansac)

# Affichage des images rectifiées
plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(imgL_rect, cmap='gray')
plt.title('Image gauche rectifiée avec épilines')
plt.subplot(122), plt.imshow(imgR_rect, cmap='gray')
plt.title('Image droite rectifiée avec épilines')
plt.show()

###### Calcul de la carte de disparité
# Assurez-vous que les deux images rectifiées ont la même taille
height, width = img1_rectified.shape
if img2_rectified.shape != img1_rectified.shape:
    img2_rectified = cv2.resize(img2_rectified, (width, height))

# Calcul de la carte de disparité avec StereoSGBM
min_disp = 0
num_disp = 16*5  # Doit être multiple de 16
block_size = 5

stereo = cv2.StereoSGBM_create(minDisparity=min_disp,
    numDisparities=num_disp,
    blockSize=block_size,
    P1=8 * 3 * block_size ** 2,
    P2=32 * 3 * block_size ** 2,
    disp12MaxDiff=1,
    uniquenessRatio=10,
    speckleWindowSize=100,
    speckleRange=32,
    preFilterCap=63,
    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY)

# Appliquer une colormap pour mieux visualiser la disparité
disparity_colormap = cv2.applyColorMap((disparity * 255).astype(np.uint8), cv2.COLORMAP_JET)

# Affichage de la carte de disparité
plt.figure(figsize=(10, 5))
plt.title("Carte de Disparité")
plt.imshow(disparity_colormap)
plt.colorbar()
plt.show()

# Fonction pour dessiner les lignes épipolaires avec correspondance de couleur correcte
def draw_corrected_epilines(img1, img2, pts1, pts2, F):
    # Trouver les épilines correspondant à certains points de l'image de droite (deuxième image)
    lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2, F)
    lines1 = lines1.reshape(-1, 3)
    img1_lines, img2_points = drawlines(img1, img2, lines1, pts1, pts2)

    # Trouver les épilines correspondant à certains points de l'image de gauche (première image)
    lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1, 1, 2), 1, F)
    lines2 = lines2.reshape(-1, 3)
    img2_lines, img1_points = drawlines(img2, img1, lines2, pts2, pts1)

    return img1_lines, img2_lines

# Tracer les droites épipolaires avec correspondance de couleur correcte
imgL_corr, imgR_corr = draw_corrected_epilines(img1, img2, inlierpts1, inlierpts2, FRansac)

# Affichage des images avec épilines corrigées
plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(imgL_corr)
plt.title('Lignes épipolaires corrigées (gauche)')
plt.subplot(122), plt.imshow(imgR_corr)
plt.title('Lignes épipolaires corrigées (droite)')
plt.show()

# Rectification des images
img1_rectified_corr = cv2.warpPerspective(img1, H1, (w1, h1))
img2_rectified_corr = cv2.warpPerspective(img2, H2, (w2, h2))

# Tracer les droites épipolaires sur les images rectifiées avec correspondance de couleur correcte
imgL_rect_corr, imgR_rect_corr = draw_corrected_epilines(img1_rectified_corr, img2_rectified_corr, inlierpts1, inlierpts2, FRansac)

# Affichage des images rectifiées avec épilines corrigées
plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(imgL_rect_corr, cmap='gray')
plt.title('Image gauche rectifiée avec épilines corrigées')
plt.subplot(122), plt.imshow(imgR_rect_corr, cmap='gray')
plt.title('Image droite rectifiée avec épilines corrigées')
plt.show()



In [ ]:
import numpy as np
from google.colab.patches import cv2_imshow
import cv2
import matplotlib.pyplot as plt

# Charger les images gauche et droite en niveaux de gris
imgLeft = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png', 0)
imgRight = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png', 0)

# Initialiser l'objet de correspondance stéréo
stereo = cv2.StereoBM_create(numDisparities=16, blockSize=5)

# Calculer l'image de disparité
disparity = stereo.compute(imgLeft, imgRight)

# Normaliser l'image pour la représentation
min_disp = disparity.min()
max_disp = disparity.max()
disparity = np.uint8(255 * (disparity - min_disp) / (max_disp - min_disp))

# Afficher les résultats côte à côte
plt.figure(figsize=(20,10))

plt.subplot(1, 3, 1)
plt.title('Image I(x,y)')
plt.imshow(imgLeft, cmap='gray')

plt.subplot(1, 3, 2)
plt.title('Carte des disparités D(x,y)')
plt.imshow(disparity, cmap='gray')

plt.subplot(1, 3, 3)
plt.title("Image I'(x', y')")
plt.imshow(imgRight, cmap='gray')

plt.show()


In [ ]:
import numpy as np
from google.colab.patches import cv2_imshow
import cv2
import matplotlib.pyplot as plt

# Charger les images gauche et droite en niveaux de gris
imgLeft = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png', 0)
imgRight = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png', 0)

# Initialiser l'objet de correspondance stéréo
stereo = cv2.StereoBM_create(numDisparities=64, blockSize=5)

# Calculer l'image de disparité
disparity = stereo.compute(imgLeft, imgRight)

# Normaliser l'image pour la représentation
min_disp = disparity.min()
max_disp = disparity.max()
disparity = np.uint8(255 * (disparity - min_disp) / (max_disp - min_disp))

# Afficher les résultats côte à côte
plt.figure(figsize=(20,10))

plt.subplot(1, 3, 1)
plt.title('Image I(x,y)')
plt.imshow(imgLeft, cmap='gray')

plt.subplot(1, 3, 2)
plt.title('Carte des disparités D(x,y)')
plt.imshow(disparity, cmap='gray')

plt.subplot(1, 3, 3)
plt.title("Image I'(x', y')")
plt.imshow(imgRight, cmap='gray')

plt.show()


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Lecture des deux images
imgL = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png', cv2.IMREAD_GRAYSCALE)  # Image gauche
imgR = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png', cv2.IMREAD_GRAYSCALE)  # Image droite

# Détection des points d'intérêt
sift = cv2.SIFT_create()
kpL, desL = sift.detectAndCompute(imgL, None)
kpR, desR = sift.detectAndCompute(imgR, None)

# Correspondances des points
bf = cv2.BFMatcher()
matches = bf.knnMatch(desL, desR, k=2)

# Appliquer le ratio test de Lowe pour filtrer les bonnes correspondances
good = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good.append(m)

# Trier les correspondances par distance
good = sorted(good, key=lambda x: x.distance)
# Conserver les 100 correspondances les plus fortes
strong_matches = good[:187]

# Préparer les points pour dessiner les lignes
ptsL = np.float32([kpL[m.queryIdx].pt for m in strong_matches])
ptsR = np.float32([kpR[m.trainIdx].pt for m in strong_matches])

# Dessiner les correspondances les plus fortes sur l'image gauche
output_img = cv2.cvtColor(imgL, cv2.COLOR_GRAY2BGR)
for ptL, ptR in zip(ptsL, ptsR):
    ptL = (int(ptL[0]), int(ptL[1]))
    ptR = (int(ptR[0]), int(ptR[1]))
    # Ajuster la longueur des lignes pour qu'elles soient plus courtes et dirigées vers la droite
    line_length = 20
    direction = (line_length, 0)  # Direction vers la droite
    end_point = (ptL[0] + direction[0], ptL[1] + direction[1])
    output_img = cv2.circle(output_img, ptL, 5, (255, 0, 255), -1)  # Points en jaune
    output_img = cv2.line(output_img, ptL, end_point, (255, 0, 255), 2)  # Lignes en jaune

# Affichage de l'image avec les correspondances
plt.figure(figsize=(10, 7))
plt.imshow(output_img)
plt.title('Correspondances les plus fortes (187) - Image gauche')
plt.axis('off')
plt.show()


In [ ]:
# Dessiner les correspondances les plus fortes entre les deux images
output_img_full = cv2.drawMatches(imgL, kpL, imgR, kpR, strong_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Affichage de l'image avec les correspondances complètes
plt.figure(figsize=(15, 10))
plt.imshow(output_img_full)
plt.title('Correspondances les plus fortes (100) - Image gauche et droite')
plt.axis('off')
plt.show()


In [ ]:
# Calcul de la matrice fondamentale avec OpenCV RANSAC
F, mask = cv2.findFundamentalMat(ptsL, ptsR, cv2.FM_RANSAC)

# Filtrer les points inliers
ptsL = ptsL[mask.ravel() == 1]
ptsR = ptsR[mask.ravel() == 1]

# Calcul des droites épipolaires
def drawlines(img1, img2, lines, pts1, pts2):
    ''' img1 - image sur laquelle on dessine les épilines pour les points dans img2
        lines - épilines correspondantes '''
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 1)
        img1 = cv2.circle(img1, (int(pt1[0]), int(pt1[1])), 3, color, -1)
        img2 = cv2.circle(img2, (int(pt2[0]), int(pt2[1])), 3, color, -1)
    return img1, img2

# Calculer les droites épipolaires pour chaque image
lines1 = cv2.computeCorrespondEpilines(ptsR.reshape(-1, 1, 2), 2, F)
lines1 = lines1.reshape(-1, 3)
lines2 = cv2.computeCorrespondEpilines(ptsL.reshape(-1, 1, 2), 1, F)
lines2 = lines2.reshape(-1, 3)

imgL_lines, imgR_lines = drawlines(imgL, imgR, lines1, ptsL, ptsR)
imgR_lines, imgL_lines = drawlines(imgR, imgL, lines2, ptsR, ptsL)

# Affichage des images avec les lignes épipolaires
plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(imgL_lines)
plt.title('Lignes épipolaires - Image gauche')
plt.subplot(122), plt.imshow(imgR_lines)
plt.title('Lignes épipolaires - Image droite')
plt.show()


In [ ]:
# Assurez-vous que les deux images ont la même taille
height, width = imgL.shape
if imgR.shape != imgL.shape:
    imgR = cv2.resize(imgR, (width, height))

# Calcul de la carte de disparité avec StereoSGBM
min_disp = 0
num_disp = 16*3  # Doit être multiple de 16
block_size = 5

stereo = cv2.StereoSGBM_create(minDisparity=min_disp,
    numDisparities=num_disp,
    blockSize=block_size,
    P1=8 * 3 * block_size ** 2,
    P2=32 * 3 * block_size ** 2,
    disp12MaxDiff=1,
    uniquenessRatio=10,
    speckleWindowSize=100,
    speckleRange=32,
    preFilterCap=63,
    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY)

disparity = stereo.compute(imgL, imgR).astype(np.float32) / 16.0

# Normaliser la disparité pour l'afficher correctement
disparity = (disparity - min_disp) / num_disp

# Appliquer une colormap pour mieux visualiser la disparité
disparity_colormap = cv2.applyColorMap((disparity * 255).astype(np.uint8), cv2.COLORMAP_JET)

# Affichage de la carte de disparité
plt.figure(figsize=(10, 5))
plt.title("Carte de Disparité")
plt.imshow(disparity_colormap)
plt.colorbar()
plt.show()


In [ ]:
# Conversion de la disparité en profondeur
focal_length = 0.2  # Longueur focale de la caméra (ajustez selon votre configuration)
baseline = 0.2  # Distance entre les caméras (en mètres, ajustez selon votre configuration)
depth = (focal_length * baseline) / (disparity + 1e-6)  # Ajouter une petite valeur pour éviter la division par zéro

# Affichage de la carte de profondeur
plt.figure(figsize=(10, 5))
plt.title("Carte de Profondeur")
plt.imshow(depth, cmap='gray')
plt.colorbar()
plt.show()


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Lecture des deux images
imgL = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png', cv2.IMREAD_GRAYSCALE)  # Image gauche
imgR = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png', cv2.IMREAD_GRAYSCALE)  # Image droite

# Détection des points d'intérêt
sift = cv2.SIFT_create()
kpL, desL = sift.detectAndCompute(imgL, None)
kpR, desR = sift.detectAndCompute(imgR, None)

# Correspondances des points
bf = cv2.BFMatcher()
matches = bf.knnMatch(desL, desR, k=2)

# Appliquer le ratio test de Lowe pour filtrer les bonnes correspondances
good = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good.append(m)

# Trier les correspondances par distance
good = sorted(good, key=lambda x: x.distance)
# Conserver les 100 correspondances les plus fortes
strong_matches = good[:100]

# Préparer les points pour dessiner les lignes
ptsL = np.float32([kpL[m.queryIdx].pt for m in strong_matches])
ptsR = np.float32([kpR[m.trainIdx].pt for m in strong_matches])

# Dessiner les correspondances les plus fortes sur l'image gauche
output_img_left = cv2.cvtColor(imgL, cv2.COLOR_GRAY2BGR)
for ptL, ptR in zip(ptsL, ptsR):
    ptL = (int(ptL[0]), int(ptL[1]))
    ptR = (int(ptR[0]), int(ptR[1]))
    # Ajuster la longueur des lignes pour qu'elles soient plus courtes et dirigées vers la droite
    line_length = 20
    direction = (line_length, 0)  # Direction vers la droite
    end_point = (ptL[0] + direction[0], ptL[1] + direction[1])
    output_img_left = cv2.circle(output_img_left, ptL, 5, (100, 0, 255), -1)  # Points en jaune
    output_img_left = cv2.line(output_img_left, ptL, end_point, (100, 0, 255), 2)  # Lignes en jaune

# Dessiner les points correspondants sur l'image droite
output_img_right = cv2.cvtColor(imgR, cv2.COLOR_GRAY2BGR)
for pt in ptsR:
    pt = (int(pt[0]), int(pt[1]))
    output_img_right = cv2.circle(output_img_right, pt, 5, (100, 0, 255), -1)  # Points en jaune

# Affichage des images originales et des correspondances
fig, axs = plt.subplots(2, 2, figsize=(12, 12))

# Afficher l'image gauche originale
axs[0, 0].imshow(cv2.cvtColor(imgL, cv2.COLOR_GRAY2RGB))
axs[0, 0].set_title('Image gauche originale')
axs[0, 0].axis('on')  # Activer les axes
axs[0, 0].grid(False)

# Afficher l'image droite originale
axs[0, 1].imshow(cv2.cvtColor(imgR, cv2.COLOR_GRAY2RGB))
axs[0, 1].set_title('Image droite originale')
axs[0, 1].axis('on')  # Activer les axes
axs[0, 1].grid(False)

# Afficher l'image gauche avec les lignes courtes
axs[1, 0].imshow(output_img_left)
axs[1, 0].set_title('Correspondances les plus fortes (100) - Image gauche')
axs[1, 0].axis('on')  # Activer les axes
axs[1, 0].grid(False)

# Afficher l'image droite avec les points correspondants
axs[1, 1].imshow(output_img_right)
axs[1, 1].set_title('Points correspondants - Image droite')
axs[1, 1].axis('on')  # Activer les axes
axs[1, 1].grid(False)

plt.tight_layout()
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
from tqdm import tqdm

# Function to perform preprocessing operations on the images
def preprocessing(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    processed_img = cv2.GaussianBlur(gray, (15,15), 0)
    return processed_img

# Function to find features in the two images
def feature_detection(img1, img2):
    processed_img1, processed_img2 = preprocessing(img1), preprocessing(img2)
    sift = cv2.SIFT_create()
    kp1, desc1 = sift.detectAndCompute(processed_img1, None)
    kp2, desc2 = sift.detectAndCompute(processed_img2, None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc1, desc2, k=2)

    # Extracting good matches
    good = []
    for m, n in matches:
        if m.distance < 0.2 * n.distance:
            good.append([m])

    left_pts = np.float32([kp1[m[0].queryIdx].pt for m in good])
    right_pts = np.float32([kp2[m[0].trainIdx].pt for m in good])
    return left_pts, right_pts

# Function to compute Fundamental Matrix given feature points in two images
def fundamental_matrix(left_pts, right_pts):
    A = []

    for i in range(8):
        src_x, src_y = left_pts[i]
        dst_x, dst_y = right_pts[i]
        A_row = np.asarray([dst_x * src_x, dst_x * src_y, dst_x, dst_y * src_x, dst_y * src_y, dst_y, src_x, src_y, 1])
        A.append(A_row)

    A = np.asarray(A)
    U, S, V = np.linalg.svd(A)
    F = np.reshape(V[-1, :], (3, 3))
    u, s, vt = np.linalg.svd(F)
    s = np.diag(s)
    s[2, 2] = 0
    F = u @ (s @ vt)
    F = F / F[-1, -1]
    F = (abs(F) > (10 ** (-3))) * F
    return F

# RANSAC function to find the best Fundamental Matrix that satisfies the epipolar constraint
def ransac(src_pts, dst_pts, iterations, threshold):
    number_of_points = len(src_pts)
    max_inlier_count = 0

    for _ in range(iterations):
        inlier = 0
        random_indices = np.random.choice(number_of_points, size=8, replace=False)
        random_src_pts = src_pts[random_indices, :]
        random_dst_pts = dst_pts[random_indices, :]

        F = fundamental_matrix(random_src_pts, random_dst_pts)

        for i in range(number_of_points):
            a, b = dst_pts[i], src_pts[i]
            a, b = np.append(a, 1).reshape((3, 1)), np.append(b, 1).reshape((3, 1))
            c = (a.T @ F) @ b

            if abs(c) <= threshold:
                inlier += 1

        if inlier > max_inlier_count:
            max_inlier_count = inlier
            best_F = F
    return best_F

# Function to compute essential matrix
def essential_matrix(F, K1, K2):
    E = (K2.T @ F) @ K1
    U, _, V = np.linalg.svd(E)
    S = [1, 1, 0]
    S = np.diag(S)
    E = np.matmul(np.matmul(U, S), V)
    return E

# Function to decompose Essential Matrix to find Rotation and Translation
def decompose_essential_matrix(E):
    W = np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]])
    U, S, V = np.linalg.svd(E)

    R_set = []
    C_set = []

    R_set.append((U @ W) @ V)
    R_set.append((U @ W) @ V)
    R_set.append((U @ W.T) @ V)
    R_set.append((U @ W.T) @ V)
    C_set.append(U[:, 2])
    C_set.append(-U[:, 2])
    C_set.append(U[:, 2])
    C_set.append(-U[:, 2])

    for i in range(4):
        if (np.linalg.det(R_set[i]) < 0):
            R_set[i] = -R_set[i]
            C_set[i] = -C_set[i]

    return R_set, C_set

# Function to find the Projection Matrix
def projection_matrix(K, R, C):
    I = np.identity(3)
    C = np.reshape(C, (3, 1))
    return (K @ (R @ np.hstack((I, -C))))

# Function to find 3D point given 2D pixel locations
def linear_triangulation(R_set, C_set, left_pts, right_pts, K1, K2):
    x3D_set = []

    for i in range(len(R_set)):
        R1, R2 = np.identity(3), R_set[i]
        C1, C2 = np.zeros((3, 1)), C_set[i].reshape(3, 1)

        P1 = projection_matrix(K1, R1, C1)
        P2 = projection_matrix(K2, R2, C2)

        p1, p2, p3 = P1
        p1_, p2_, p3_ = P2

        p1, p2, p3 = p1.reshape(1, -1), p2.reshape(1, -1), p3.reshape(1, -1)
        p1_, p2_, p3_ = p1_.reshape(1, -1), p2_.reshape(1, -1), p3_.reshape(1, -1)

        x3D = []

        for left_pt, right_pt in zip(left_pts, right_pts):
            x, y = left_pt
            x_, y_ = right_pt
            A = np.vstack((y * p3 - p2, p1 - x * p3, y_ * p3_ - p2_, p1_ - x_ * p3_))
            _, _, Vt = np.linalg.svd(A)
            X = Vt[-1]
            X = X / X[-1]
            x3D.append(X[:3])

        x3D_set.append(x3D)
    return x3D_set

# Function to find best camera pose configuration out of the four options
def disambiguate_camera_pose(R_set, C_set, x3D_set):
    best_i = 0
    max_positive_depths = 0

    for i in range(len(R_set)):
        n_positive_depths = 0
        R, C = R_set[i], C_set[i].reshape(-1, 1)
        r3 = R[2].reshape(1, -1)
        x3D = x3D_set[i]

        for X in x3D:
            X = X.reshape(-1, 1)
            if r3 @ (X - C) > 0 and X[2] > 0:
                n_positive_depths += 1

        if n_positive_depths > max_positive_depths:
            best_i = i
            max_positive_depths = n_positive_depths

    R, C = R_set[best_i], C_set[best_i]
    return R, C

# Function to find start and end co-ordinates given equation of an epiline
def epiline_coordinates(lines, img):
    lines = lines.reshape(-1, 3)
    c = img.shape[1]
    co_ordinates = []

    for line in lines:
        x0, y0 = map(int, [0, -line[2] / line[1]])
        x1, y1 = map(int, [c, -(line[2] + line[0] * c) / line[1]])
        co_ordinates.append([[x0, y0], [x1, y1]])

    return co_ordinates

# Function to draw epilines on the two corresponding images
# Continue drawing epilines on the two corresponding images
def draw_epilines(l_epiline_coords, r_epiline_coords, left_pts, right_pts, img1, img2):
    img1_copy = copy.deepcopy(img1)
    img2_copy = copy.deepcopy(img2)

    for l_epiline_coord, r_epiline_coord, left_pt, right_pt in zip(l_epiline_coords, r_epiline_coords, np.int32(left_pts), np.int32(right_pts)):
        color = tuple(np.random.randint(0,255,3).tolist())
        img1_copy = cv2.line(img1_copy, l_epiline_coord[0], l_epiline_coord[1], color, 2)
        img1_copy = cv2.circle(img1_copy, (left_pt[0], left_pt[1]), 7, color, -1)
        img2_copy = cv2.line(img2_copy, r_epiline_coord[0], r_epiline_coord[1], color, 2)
        img2_copy = cv2.circle(img2_copy, (right_pt[0], right_pt[1]), 7, color, -1)

    return img1_copy, img2_copy

# Load images
img1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png')
img2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png')

# Detect features and compute fundamental matrix using RANSAC
left_pts, right_pts = feature_detection(img1, img2)
F = ransac(left_pts, right_pts, 1500, 0.02)

# Compute essential matrix and decompose it to find rotation and translation
K1 = np.array([[1758.23, 0, 977.42], [0, 1758.23, 552.15], [0, 0, 1]])
K2 = np.array([[1758.23, 0, 977.42], [0, 1758.23, 552.15], [0, 0, 1]])
E = essential_matrix(F, K1, K2)
R_set, C_set = decompose_essential_matrix(E)
x3D_set = linear_triangulation(R_set, C_set, left_pts, right_pts, K1, K2)
R, T = disambiguate_camera_pose(R_set, C_set, x3D_set)

# Compute epilines
l_epilines = cv2.computeCorrespondEpilines(right_pts.reshape(-1, 1, 2), 2, F)
r_epilines = cv2.computeCorrespondEpilines(left_pts.reshape(-1, 1, 2), 1, F)
l_epiline_coords = epiline_coordinates(l_epilines, img1)
r_epiline_coords = epiline_coordinates(r_epilines, img2)

# Draw epilines on the images
img1_with_epilines, img2_with_epilines = draw_epilines(l_epiline_coords, r_epiline_coords, left_pts, right_pts, img1, img2)

# Display the images with epilines
plt.figure(figsize=(15, 10))
plt.subplot(121), plt.imshow(cv2.cvtColor(img1_with_epilines, cv2.COLOR_BGR2RGB))
plt.title('Lignes épipolaires - Image gauche')
plt.subplot(122), plt.imshow(cv2.cvtColor(img2_with_epilines, cv2.COLOR_BGR2RGB))
plt.title('Lignes épipolaires - Image droite')
plt.show()

# Stereo Rectification
h1, w1 = img1.shape[:2]
_, H1, H2 = cv2.stereoRectifyUncalibrated(np.float32(left_pts), np.float32(right_pts), F, imgSize=(w1, h1))

# Rectify the images
img1_rectified = cv2.warpPerspective(img1, H1, (w1, h1))
img2_rectified = cv2.warpPerspective(img2, H2, (w1, h1))

# Compute disparity map
a_gray = cv2.cvtColor(img1_rectified, cv2.COLOR_BGR2GRAY)
b_gray = cv2.cvtColor(img2_rectified, cv2.COLOR_BGR2GRAY)
a_gray = a_gray.astype(float)
b_gray = b_gray.astype(float)
h, w = a_gray.shape

disparity_map = np.zeros((h, w))
BLOCK_SIZE = 7
SEARCH_BLOCK_SIZE = 56

for y in tqdm(range(BLOCK_SIZE, h - BLOCK_SIZE)):
    for x in range(BLOCK_SIZE, w - BLOCK_SIZE):
        block_left = a_gray[y:y + BLOCK_SIZE, x:x + BLOCK_SIZE]
        min_index = compare_blocks(y, x, block_left, b_gray, BLOCK_SIZE, SEARCH_BLOCK_SIZE)
        disparity_map[y, x] = abs(min_index[1] - x)

disparity = np.uint8(disparity_map * 255 / np.max(disparity_map))
heatmap_disparity = cv2.applyColorMap(disparity, cv2.COLORMAP_JET)

# Compute depth map
f = K1[0, 0]
baseline = 88.39
depth = (baseline * f) / (disparity + 1e-15)
depth[depth > 30000] = 30000
depth_map = np.uint8(depth * 255 / np.max(depth))
heatmap_depth = cv2.applyColorMap(depth_map, cv2.COLORMAP_JET)

# Display the rectified images, disparity map, and depth map
plt.figure(figsize=(20, 10))

# Display rectified images
plt.subplot(2, 3, 1)
plt.imshow(cv2.cvtColor(img1_rectified, cv2.COLOR_BGR2RGB))
plt.title('Image gauche rectifiée')
plt.axis('on')
plt.grid(True)

plt.subplot(2, 3, 2)
plt.imshow(cv2.cvtColor(img2_rectified, cv2.COLOR_BGR2RGB))
plt.title('Image droite rectifiée')
plt.axis('on')
plt.grid(True)

# Display disparity map
plt.subplot(2, 3, 3)
plt.imshow(heatmap_disparity)
plt.title('Carte de disparité')
plt.colorbar()
plt.axis('on')
plt.grid(True)

# Display depth map
plt.subplot(2, 3, 4)
plt.imshow(heatmap_depth)
plt.title('Carte de profondeur')
plt.colorbar()
plt.axis('on')
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import copy
from tqdm import tqdm

# Function to perform preprocessing operations on the images
def preprocessing(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    processed_img = cv2.GaussianBlur(gray, (15, 15), 0)
    return processed_img

# Function to find features in the two images
def feature_detection(img1, img2):
    processed_img1, processed_img2 = preprocessing(img1), preprocessing(img2)
    sift = cv2.SIFT_create()
    kp1, desc1 = sift.detectAndCompute(processed_img1, None)
    kp2, desc2 = sift.detectAndCompute(processed_img2, None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc1, desc2, k=2)

    # Extracting good matches
    good = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good.append([m])

    left_pts = np.float32([kp1[m[0].queryIdx].pt for m in good])
    right_pts = np.float32([kp2[m[0].trainIdx].pt for m in good])
    return left_pts, right_pts, kp1, kp2, good

# Function to compute Fundamental Matrix given feature points in two images
def fundamental_matrix(left_pts, right_pts):
    A = []

    for i in range(8):
        src_x, src_y = left_pts[i]
        dst_x, dst_y = right_pts[i]
        A_row = np.asarray([dst_x * src_x, dst_x * src_y, dst_x, dst_y * src_x, dst_y * src_y, dst_y, src_x, src_y, 1])
        A.append(A_row)

    A = np.asarray(A)
    U, S, V = np.linalg.svd(A)
    F = np.reshape(V[-1, :], (3, 3))
    u, s, vt = np.linalg.svd(F)
    s = np.diag(s)
    s[2, 2] = 0
    F = u @ (s @ vt)
    F = F / F[-1, -1]
    F = (abs(F) > (10 ** (-3))) * F
    return F

# RANSAC function to find the best Fundamental Matrix that satisfies the epipolar constraint
def ransac(src_pts, dst_pts, iterations, threshold):
    number_of_points = len(src_pts)
    max_inlier_count = 0

    for _ in range(iterations):
        inlier = 0
        random_indices = np.random.choice(number_of_points, size=8, replace=False)
        random_src_pts = src_pts[random_indices, :]
        random_dst_pts = dst_pts[random_indices, :]

        F = fundamental_matrix(random_src_pts, random_dst_pts)

        for i in range(number_of_points):
            a, b = dst_pts[i], src_pts[i]
            a, b = np.append(a, 1).reshape((3, 1)), np.append(b, 1).reshape((3, 1))
            c = (a.T @ F) @ b

            if abs(c) <= threshold:
                inlier += 1

        if inlier > max_inlier_count:
            max_inlier_count = inlier
            best_F = F
    return best_F

# Function to compute essential matrix
def essential_matrix(F, K1, K2):
    E = (K2.T @ F) @ K1
    U, _, V = np.linalg.svd(E)
    S = [1, 1, 0]
    S = np.diag(S)
    E = np.matmul(np.matmul(U, S), V)
    return E

# Function to decompose Essential Matrix to find Rotation and Translation
def decompose_essential_matrix(E):
    W = np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]])
    U, S, V = np.linalg.svd(E)

    R_set = []
    C_set = []

    R_set.append((U @ W) @ V)
    R_set.append((U @ W) @ V)
    R_set.append((U @ W.T) @ V)
    R_set.append((U @ W.T) @ V)
    C_set.append(U[:, 2])
    C_set.append(-U[:, 2])
    C_set.append(U[:, 2])
    C_set.append(-U[:, 2])

    for i in range(4):
        if (np.linalg.det(R_set[i]) < 0):
            R_set[i] = -R_set[i]
            C_set[i] = -C_set[i]

    return R_set, C_set

# Function to find the Projection Matrix
def projection_matrix(K, R, C):
    I = np.identity(3)
    C = np.reshape(C, (3, 1))
    return (K @ (R @ np.hstack((I, -C))))

# Function to find 3D point given 2D pixel locations
def linear_triangulation(R_set, C_set, left_pts, right_pts, K1, K2):
    x3D_set = []

    for i in range(len(R_set)):
        R1, R2 = np.identity(3), R_set[i]
        C1, C2 = np.zeros((3, 1)), C_set[i].reshape(3, 1)

        P1 = projection_matrix(K1, R1, C1)
        P2 = projection_matrix(K2, R2, C2)

        p1, p2, p3 = P1
        p1_, p2_, p3_ = P2

        p1, p2, p3 = p1.reshape(1, -1), p2.reshape(1, -1), p3.reshape(1, -1)
        p1_, p2_, p3_ = p1_.reshape(1, -1), p2_.reshape(1, -1), p3_.reshape(1, -1)

        x3D = []

        for left_pt, right_pt in zip(left_pts, right_pts):
            x, y = left_pt
            x_, y_ = right_pt
            A = np.vstack((y * p3 - p2, p1 - x * p3, y_ * p3_ - p2_, p1_ - x_ * p3_))
            _, _, Vt = np.linalg.svd(A)
            X = Vt[-1]
            X = X / X[-1]
            x3D.append(X[:3])

        x3D_set.append(x3D)
    return x3D_set

# Function to find best camera pose configuration out of the four options
def disambiguate_camera_pose(R_set, C_set, x3D_set):
    best_i = 0
    max_positive_depths = 0

    for i in range(len(R_set)):
        n_positive_depths = 0
        R, C = R_set[i], C_set[i].reshape(-1, 1)
        r3 = R[2].reshape(1, -1)
        x3D = x3D_set[i]

        for X in x3D:
            X = X.reshape(-1, 1)
            if r3 @ (X - C) > 0 and X[2] > 0:
                n_positive_depths += 1

        if n_positive_depths > max_positive_depths:
            best_i = i
            max_positive_depths = n_positive_depths

    R, C = R_set[best_i], C_set[best_i]
    return R, C

# Function to find start and end coordinates given equation of an epiline
def epiline_coordinates(lines, img):
    lines = lines.reshape(-1, 3)
    c = img.shape[1]
    coordinates = []

    for line in lines:
        x0, y0 = map(int, [0, -line[2] / line[1]])
        x1, y1 = map(int, [c, -(line[2] + line[0] * c) / line[1]])
        coordinates.append([[x0, y0], [x1, y1]])

    return coordinates

# Function to draw epilines on the two corresponding images
def draw_epilines(l_epiline_coords, r_epiline_coords, left_pts, right_pts, img1, img2):
    img1_copy = copy.deepcopy(img1)
    img2_copy = copy.deepcopy(img2)

    for l_epiline_coord, r_epiline_coord, left_pt, right_pt in zip(l_epiline_coords, r_epiline_coords, np.int32(left_pts), np.int32(right_pts)):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        img1_copy = cv2.line(img1_copy, l_epiline_coord[0], l_epiline_coord[1], color, 2)
        img1_copy = cv2.circle(img1_copy, (left_pt[0], left_pt[1]), 7, color, -1)
        img2_copy = cv2.line(img2_copy, r_epiline_coord[0], r_epiline_coord[1], color, 2)
        img2_copy = cv2.circle(img2_copy, (right_pt[0], right_pt[1]), 7, color, -1)

    return img1_copy, img2_copy

# Load images
img1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png')
img2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png')

# Detect features and compute fundamental matrix using RANSAC
left_pts, right_pts, kp1, kp2, good = feature_detection(img1, img2)
F = ransac(left_pts, right_pts, 1500, 0.02)

# Compute essential matrix and decompose it to find rotation and translation
K1 = np.array([[1758.23, 0, 977.42], [0, 1758.23, 552.15], [0, 0, 1]])
K2 = np.array([[1758.23, 0, 977.42], [0, 1758.23, 552.15], [0, 0, 1]])
E = essential_matrix(F, K1, K2)
R_set, C_set = decompose_essential_matrix(E)
x3D_set = linear_triangulation(R_set, C_set, left_pts, right_pts, K1, K2)
R, T = disambiguate_camera_pose(R_set, C_set, x3D_set)

# Compute epilines
l_epilines = cv2.computeCorrespondEpilines(right_pts.reshape(-1, 1, 2), 2, F)
r_epilines = cv2.computeCorrespondEpilines(left_pts.reshape(-1, 1, 2), 1, F)
l_epiline_coords = epiline_coordinates(l_epilines, img1)
r_epiline_coords = epiline_coordinates(r_epilines, img2)

# Draw epilines on the images
img1_with_epilines, img2_with_epilines = draw_epilines(l_epiline_coords, r_epiline_coords, left_pts, right_pts, img1, img2)

# Concatenate images horizontally for display
img_pair_with_epilines = np.hstack((img1_with_epilines, img2_with_epilines))

# Display the images with epilines
plt.figure(figsize=(15, 10))
plt.imshow(cv2.cvtColor(img_pair_with_epilines, cv2.COLOR_BGR2RGB))
plt.title('Epipolar lines corresponding to the obtained matching features')
plt.axis('off')
plt.show()

# Stereo Rectification
h1, w1 = img1.shape[:2]
_, H1, H2 = cv2.stereoRectifyUncalibrated(np.float32(left_pts), np.float32(right_pts), F, imgSize=(w1, h1))

# Rectify the images
img1_rectified = cv2.warpPerspective(img1, H1, (w1, h1))
img2_rectified = cv2.warpPerspective(img2, H2, (w1, h1))

# Compute rectified epilines
l_epilines_rect = cv2.computeCorrespondEpilines(right_pts.reshape(-1, 1, 2), 2, F)
r_epilines_rect = cv2.computeCorrespondEpilines(left_pts.reshape(-1, 1, 2), 1, F)
l_epiline_coords_rect = epiline_coordinates(l_epilines_rect, img1_rectified)
r_epiline_coords_rect = epiline_coordinates(r_epilines_rect, img2_rectified)

# Draw epilines on the rectified images
img1_with_epilines_rect, img2_with_epilines_rect = draw_epilines(l_epiline_coords_rect, r_epiline_coords_rect, left_pts, right_pts, img1_rectified, img2_rectified)

# Concatenate rectified images horizontally for display
img_pair_with_epilines_rect = np.hstack((img1_with_epilines_rect, img2_with_epilines_rect))

# Display the rectified images with epilines
plt.figure(figsize=(15, 10))
plt.imshow(cv2.cvtColor(img_pair_with_epilines_rect, cv2.COLOR_BGR2RGB))
plt.title('Rectified epipolar lines')
plt.axis('off')
plt.show()

# Compute disparity map
a_gray = cv2.cvtColor(img1_rectified, cv2.COLOR_BGR2GRAY)
b_gray = cv2.cvtColor(img2_rectified, cv2.COLOR_BGR2GRAY)
a_gray = a_gray.astype(float)
b_gray = b_gray.astype(float)
h, w = a_gray.shape

disparity_map = np.zeros((h, w))
BLOCK_SIZE = 7
SEARCH_BLOCK_SIZE = 56

for y in tqdm(range(BLOCK_SIZE, h - BLOCK_SIZE)):
    for x in range(BLOCK_SIZE, w - BLOCK_SIZE):
        block_left = a_gray[y:y + BLOCK_SIZE, x:x + BLOCK_SIZE]
        min_index = compare_blocks(y, x, block_left, b_gray, BLOCK_SIZE, SEARCH_BLOCK_SIZE)
        disparity_map[y, x] = abs(min_index[1] - x)

# Normalize disparity map for display
disparity = np.uint8(disparity_map * 255 / np.max(disparity_map))

# Compute depth map
f = K1[0, 0]
baseline = 99.39
depth = (baseline * f) / (disparity_map + 1e-15)
depth[depth > 30000] = 30000
depth_map = np.uint8(depth * 255 / np.max(depth))

# Display the results
plt.figure(figsize=(20, 15))

# Display the disparity map
plt.subplot(2, 2, 1)
plt.imshow(disparity, cmap='gray')
plt.title('Disparity Map')
plt.colorbar()

# Display the depth map
plt.subplot(2, 2, 2)
plt.imshow(depth_map, cmap='gray')
plt.title('Depth Map')
plt.colorbar()

plt.tight_layout()
plt.show()



In [ ]:
!cd /content && git clone 'https://github.com/verlab/accelerated_features.git'
%cd /content/accelerated_features

In [ ]:
import cv2
import numpy as np

def warp_corners_and_draw_matches(ref_points, dst_points, img1, img2):
    # Calculate the Homography matrix
    H, mask = cv2.findHomography(ref_points, dst_points, cv2.USAC_MAGSAC, 3.5, maxIters=1_000, confidence=0.999)
    mask = mask.flatten()

    # Get corners of the first image (image1)
    h, w = img1.shape[:2]
    corners_img1 = np.array([[0, 0], [w-1, 0], [w-1, h-1], [0, h-1]], dtype=np.float32).reshape(-1, 1, 2)

    # Warp corners to the second image (image2) space
    warped_corners = cv2.perspectiveTransform(corners_img1, H)

    # Draw the warped corners in image2
    img2_with_corners = img2.copy()
    for i in range(len(warped_corners)):
        start_point = tuple(warped_corners[i-1][0].astype(int))
        end_point = tuple(warped_corners[i][0].astype(int))
        cv2.line(img2_with_corners, start_point, end_point, (0, 255, 0), 5)  # Using solid green for corners

    # Prepare keypoints and matches for drawMatches function
    keypoints1 = [cv2.KeyPoint(p[0], p[1], 10) for p in ref_points]
    keypoints2 = [cv2.KeyPoint(p[0], p[1], 10) for p in dst_points]
    matches = [cv2.DMatch(i,i,0) for i in range(len(mask)) if mask[i]]

    # Draw inlier matches
    img_matches = cv2.drawMatches(img1, keypoints1, img2_with_corners, keypoints2, matches, None,
                                  matchColor=(0, 255, 255), flags=2)

    return img_matches


In [ ]:
import numpy as np
import os
import torch
import tqdm
import cv2
import matplotlib.pyplot as plt

from modules.xfeat import XFeat

xfeat = XFeat()

#Load some example images
im1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png')
im2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png')

#Use out-of-the-box function for extraction + MNN matching
mkpts_0, mkpts_1 = xfeat.match_xfeat(im1, im2, top_k = 350)

canvas = warp_corners_and_draw_matches(mkpts_0, mkpts_1, im1, im2)
plt.figure(figsize=(12,12))
plt.imshow(canvas[..., ::-1]), plt.show()

In [ ]:
#Use out-of-the-box function for extraction + coarse-to-fine matching
mkpts_0, mkpts_1 = xfeat.match_xfeat_star(im1, im2, top_k = 150)

canvas = warp_corners_and_draw_matches(mkpts_0, mkpts_1, im1, im2)
plt.figure(figsize=(12,12))
plt.imshow(canvas[..., ::-1]), plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to perform preprocessing operations on the images
def preprocessing(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    processed_img = cv2.GaussianBlur(gray, (5, 5), 0)
    return processed_img

# Function to find features in the two images
def feature_detection(img1, img2):
    processed_img1, processed_img2 = preprocessing(img1), preprocessing(img2)
    sift = cv2.SIFT_create()
    kp1, desc1 = sift.detectAndCompute(processed_img1, None)
    kp2, desc2 = sift.detectAndCompute(processed_img2, None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc1, desc2, k=2)

    # Extracting good matches
    good = []
    for m, n in matches:
        if m.distance < 0.2 * n.distance:
            good.append(m)

    left_pts = np.float32([kp1[m.queryIdx].pt for m in good])
    right_pts = np.float32([kp2[m.trainIdx].pt for m in good])
    return left_pts, right_pts, kp1, kp2, good

# Function to compute Fundamental Matrix given feature points in two images
def fundamental_matrix(left_pts, right_pts):
    F, mask = cv2.findFundamentalMat(left_pts, right_pts, cv2.FM_RANSAC)
    return F, mask

# Function to draw epilines on the images
def draw_epilines(img1, img2, lines, pts1, pts2):
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 1)
        img1 = cv2.circle(img1, (int(pt1[0]), int(pt1[1])), 5, color, -1)
        img2 = cv2.circle(img2, (int(pt2[0]), int(pt2[1])), 5, color, -1)
    return img1, img2

# Load images
img1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png')
img2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png')

# Detect features and compute fundamental matrix using RANSAC
left_pts, right_pts, kp1, kp2, good = feature_detection(img1, img2)
F, mask = fundamental_matrix(left_pts, right_pts)

# Select inlier points
left_pts = left_pts[mask.ravel() == 1]
right_pts = right_pts[mask.ravel() == 1]

# Compute epilines for both images
lines1 = cv2.computeCorrespondEpilines(right_pts.reshape(-1, 1, 2), 2, F)
lines1 = lines1.reshape(-1, 3)
img1_with_lines, _ = draw_epilines(preprocessing(img1), preprocessing(img2), lines1, left_pts, right_pts)

lines2 = cv2.computeCorrespondEpilines(left_pts.reshape(-1, 1, 2), 1, F)
lines2 = lines2.reshape(-1, 3)
img2_with_lines, _ = draw_epilines(preprocessing(img2), preprocessing(img1), lines2, right_pts, left_pts)

# Display original images with epilines
plt.figure(figsize=(20, 10))
plt.subplot(121), plt.imshow(img1_with_lines), plt.title('Image gauche avec lignes épipolaires')
plt.subplot(122), plt.imshow(img2_with_lines), plt.title('Image droite avec lignes épipolaires')
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to perform preprocessing operations on the images
def preprocessing(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    processed_img = cv2.GaussianBlur(gray, (5, 5), 0)
    return processed_img

# Function to find features in the two images
def feature_detection(img1, img2):
    processed_img1, processed_img2 = preprocessing(img1), preprocessing(img2)
    sift = cv2.SIFT_create()
    kp1, desc1 = sift.detectAndCompute(processed_img1, None)
    kp2, desc2 = sift.detectAndCompute(processed_img2, None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc1, desc2, k=2)

    # Extracting good matches
    good = []
    for m, n in matches:
        if m.distance < 0.1 * n.distance:
            good.append(m)

    left_pts = np.float32([kp1[m.queryIdx].pt for m in good])
    right_pts = np.float32([kp2[m.trainIdx].pt for m in good])
    return left_pts, right_pts, kp1, kp2, good

# Function to compute Fundamental Matrix given feature points in two images
def fundamental_matrix(left_pts, right_pts):
    F, mask = cv2.findFundamentalMat(left_pts, right_pts, cv2.FM_RANSAC)
    return F, mask

# Function to draw epilines on the images
def draw_epilines(img1, img2, lines, pts1, pts2):
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 2)
        img1 = cv2.circle(img1, (int(pt1[0]), int(pt1[1])), 3, color, -1)
        img2 = cv2.circle(img2, (int(pt2[0]), int(pt2[1])), 3, color, -1)
    return img1, img2

# Load images
img1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png')
img2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png')

# Detect features and compute fundamental matrix using RANSAC
left_pts, right_pts, kp1, kp2, good = feature_detection(img1, img2)
F, mask = fundamental_matrix(left_pts, right_pts)

# Select inlier points
left_pts = left_pts[mask.ravel() == 1]
right_pts = right_pts[mask.ravel() == 1]

# Combine images side by side
img_combined = np.hstack((img1, img2))
gray_combined = cv2.cvtColor(img_combined, cv2.COLOR_BGR2GRAY)

# Compute epilines for both images
lines1 = cv2.computeCorrespondEpilines(right_pts.reshape(-1, 1, 2), 2, F)
lines1 = lines1.reshape(-1, 3)
img1_with_lines, _ = draw_epilines(preprocessing(img1), preprocessing(img2), lines1, left_pts, right_pts)

lines2 = cv2.computeCorrespondEpilines(left_pts.reshape(-1, 1, 2), 1, F)
lines2 = lines2.reshape(-1, 3)
img2_with_lines, _ = draw_epilines(preprocessing(img2), preprocessing(img1), lines2, right_pts, left_pts)

# Combine images with epilines side by side
img_combined_with_lines = np.hstack((img1_with_lines, img2_with_lines))

# Display combined image with epilines
plt.figure(figsize=(20, 10))
plt.imshow(img_combined_with_lines)
plt.title('Combined Image with Epipolar Lines')
plt.axis('off')
plt.show()

# Stereo rectification
h1, w1 = img1.shape[:2]
_, H1, H2 = cv2.stereoRectifyUncalibrated(np.float32(left_pts), np.float32(right_pts), F, imgSize=(w1, h1))

# Rectify the images
img1_rectified = cv2.warpPerspective(img1, H1, (w1, h1))
img2_rectified = cv2.warpPerspective(img2, H2, (w1, h1))

# Combine rectified images side by side
img_combined_rectified = np.hstack((img1_rectified, img2_rectified))

# Compute epilines for the rectified images
left_pts_rect = cv2.perspectiveTransform(left_pts.reshape(-1, 1, 2), H1).reshape(-1, 2)
right_pts_rect = cv2.perspectiveTransform(right_pts.reshape(-1, 1, 2), H2).reshape(-1, 2)

lines1_rect = cv2.computeCorrespondEpilines(right_pts_rect.reshape(-1, 1, 2), 2, F)
lines1_rect = lines1_rect.reshape(-1, 3)
img1_rect_with_lines, _ = draw_epilines(preprocessing(img1_rectified), preprocessing(img2_rectified), lines1_rect, left_pts_rect, right_pts_rect)

lines2_rect = cv2.computeCorrespondEpilines(left_pts_rect.reshape(-1, 1, 2), 1, F)
lines2_rect = lines2_rect.reshape(-1, 3)
img2_rect_with_lines, _ = draw_epilines(preprocessing(img2_rectified), preprocessing(img1_rectified), lines2_rect, right_pts_rect, left_pts_rect)

# Combine rectified images with epilines side by side
img_combined_rect_with_lines = np.hstack((img1_rect_with_lines, img2_rect_with_lines))

# Display rectified combined images with epilines
plt.figure(figsize=(20, 10))
plt.imshow(img_combined_rect_with_lines)
plt.title('Rectified Combined Image with Epipolar Lines')
plt.axis('off')
plt.show()

# Compute disparity map
a_gray = cv2.cvtColor(img1_rectified, cv2.COLOR_BGR2GRAY)
b_gray = cv2.cvtColor(img2_rectified, cv2.COLOR_BGR2GRAY)
stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)
disparity = stereo.compute(a_gray, b_gray)

# Normalize disparity map for display
disparity = cv2.normalize(disparity, disparity, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
disparity = np.uint8(disparity)

# Display disparity map
plt.figure(figsize=(10, 5))
plt.imshow(disparity, cmap='gray')
plt.title('Disparity Map')
plt.axis('off')
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to find features in the two images
def feature_detection(img1, img2):
    sift = cv2.SIFT_create()
    kp1, desc1 = sift.detectAndCompute(img1, None)
    kp2, desc2 = sift.detectAndCompute(img2, None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc1, desc2, k=2)

    # Extracting good matches
    good = []
    for m, n in matches:
        if m.distance < 0.1 * n.distance:
            good.append(m)

    left_pts = np.float32([kp1[m.queryIdx].pt for m in good])
    right_pts = np.float32([kp2[m.trainIdx].pt for m in good])
    return left_pts, right_pts, kp1, kp2, good

# Function to compute Fundamental Matrix given feature points in two images
def fundamental_matrix(left_pts, right_pts):
    F, mask = cv2.findFundamentalMat(left_pts, right_pts, cv2.FM_RANSAC)
    return F, mask

# Function to draw epilines on the images
def draw_epilines(img1, img2, lines, pts1, pts2):
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 2)
        img1 = cv2.circle(img1, (int(pt1[0]), int(pt1[1])), 3, color, -1)
        img2 = cv2.circle(img2, (int(pt2[0]), int(pt2[1])), 3, color, -1)
    return img1, img2

# Load images
img1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png', cv2.IMREAD_GRAYSCALE)

# Detect features and compute fundamental matrix using RANSAC
left_pts, right_pts, kp1, kp2, good = feature_detection(img1, img2)
F, mask = fundamental_matrix(left_pts, right_pts)

# Select inlier points
left_pts = left_pts[mask.ravel() == 1]
right_pts = right_pts[mask.ravel() == 1]

# Compute epilines for both images
lines1 = cv2.computeCorrespondEpilines(right_pts.reshape(-1, 1, 2), 2, F)
lines1 = lines1.reshape(-1, 3)
img1_with_lines, _ = draw_epilines(img1, img2, lines1, left_pts, right_pts)

lines2 = cv2.computeCorrespondEpilines(left_pts.reshape(-1, 1, 2), 1, F)
lines2 = lines2.reshape(-1, 3)
img2_with_lines, _ = draw_epilines(img2, img1, lines2, right_pts, left_pts)

# Combine images with epilines side by side
img_combined_with_lines = np.hstack((img1_with_lines, img2_with_lines))

# Display combined image with epilines
plt.figure(figsize=(20, 10))
plt.imshow(img_combined_with_lines)
plt.title('Combined Image with Epipolar Lines (Without Rectification)')
plt.axis('off')
plt.savefig('/content/drive/MyDrive/COMPUTER_VISION/')
plt.show()

# Stereo rectification
h1, w1 = img1.shape[:2]
_, H1, H2 = cv2.stereoRectifyUncalibrated(np.float32(left_pts), np.float32(right_pts), F, imgSize=(w1, h1))

# Rectify the images
img1_rectified = cv2.warpPerspective(img1, H1, (w1, h1))
img2_rectified = cv2.warpPerspective(img2, H2, (w1, h1))

# Combine rectified images side by side
img_combined_rectified = np.hstack((img1_rectified, img2_rectified))

# Compute epilines for the rectified images
left_pts_rect = cv2.perspectiveTransform(left_pts.reshape(-1, 1, 2), H1).reshape(-1, 2)
right_pts_rect = cv2.perspectiveTransform(right_pts.reshape(-1, 1, 2), H2).reshape(-1, 2)

lines1_rect = cv2.computeCorrespondEpilines(right_pts_rect.reshape(-1, 1, 2), 2, F)
lines1_rect = lines1_rect.reshape(-1, 3)
img1_rect_with_lines, _ = draw_epilines(img1_rectified, img2_rectified, lines1_rect, left_pts_rect, right_pts_rect)

lines2_rect = cv2.computeCorrespondEpilines(left_pts_rect.reshape(-1, 1, 2), 1, F)
lines2_rect = lines2_rect.reshape(-1, 3)
img2_rect_with_lines, _ = draw_epilines(img2_rectified, img1_rectified, lines2_rect, right_pts_rect, left_pts_rect)

# Combine rectified images with epilines side by side
img_combined_rect_with_lines = np.hstack((img1_rect_with_lines, img2_rect_with_lines))

# Display rectified combined images with epilines
plt.figure(figsize=(20, 10))
plt.imshow(img_combined_rect_with_lines)
plt.title('Rectified Combined Image with Epipolar Lines')
plt.axis('off')
plt.savefig('/content/drive/MyDrive/COMPUTER_VISION/')
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Lecture des deux images
img1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png')
img2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png')

# Conversion en niveaux de gris
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Détection des points d'intérêt
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(gray1, None)
kp2, des2 = sift.detectAndCompute(gray2, None)

# Appariement des points d'intérêt
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)

# Extraction des points appariés
pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])

# Calcul de la matrice fondamentale
F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)

# Sélection des bons appariements
pts1 = pts1[mask.ravel() == 1]
pts2 = pts2[mask.ravel() == 1]

# Affichage des lignes épipolaires avant rectification
def draw_epilines(img1, img2, lines, pts1, pts2):
    r, c = img1.shape[:2]
    img1_color = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2_color = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1_color = cv2.line(img1_color, (x0, y0), (x1, y1), color, 1)
        img1_color = cv2.circle(img1_color, tuple(map(int, pt1)), 5, color, -1)
        img2_color = cv2.circle(img2_color, tuple(map(int, pt2)), 5, color, -1)
    return img1_color, img2_color

# Calcul des lignes épipolaires
lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2, F)
lines1 = lines1.reshape(-1, 3)
img1_epilines, img2_epilines = draw_epilines(gray1, gray2, lines1, pts1, pts2)

lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1, 1, 2), 1, F)
lines2 = lines2.reshape(-1, 3)
img2_epilines, img1_epilines = draw_epilines(gray2, gray1, lines2, pts2, pts1)

# Affichage des images avec lignes épipolaires
plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(img1_epilines)
plt.subplot(122), plt.imshow(img2_epilines)
plt.suptitle('Lignes épipolaires avant rectification')
plt.show()

# Rectification des images
h1, w1 = gray1.shape
h2, w2 = gray2.shape

_, H1, H2 = cv2.stereoRectifyUncalibrated(np.float32(pts1), np.float32(pts2), F, imgSize=(w1, h1))

img1_rectified = cv2.warpPerspective(img1, H1, (w1, h1))
img2_rectified = cv2.warpPerspective(img2, H2, (w2, h2))

gray1_rectified = cv2.warpPerspective(gray1, H1, (w1, h1))
gray2_rectified = cv2.warpPerspective(gray2, H2, (w2, h2))

# Recalcul des points appariés après rectification
kp1_rectified, des1_rectified = sift.detectAndCompute(gray1_rectified, None)
kp2_rectified, des2_rectified = sift.detectAndCompute(gray2_rectified, None)
matches_rectified = bf.match(des1_rectified, des2_rectified)
matches_rectified = sorted(matches_rectified, key=lambda x: x.distance)

pts1_rectified = np.float32([kp1_rectified[m.queryIdx].pt for m in matches_rectified])
pts2_rectified = np.float32([kp2_rectified[m.trainIdx].pt for m in matches_rectified])

# Affichage des lignes épipolaires après rectification
lines1_rectified = cv2.computeCorrespondEpilines(pts2_rectified.reshape(-1, 1, 2), 2, F)
lines1_rectified = lines1_rectified.reshape(-1, 3)
img1_rect_epilines, img2_rect_epilines = draw_epilines(gray1_rectified, gray2_rectified, lines1_rectified, pts1_rectified, pts2_rectified)

lines2_rectified = cv2.computeCorrespondEpilines(pts1_rectified.reshape(-1, 1, 2), 1, F)
lines2_rectified = lines2_rectified.reshape(-1, 3)
img2_rect_epilines, img1_rect_epilines = draw_epilines(gray2_rectified, gray1_rectified, lines2_rectified, pts2_rectified, pts1_rectified)

# Affichage des images rectifiées avec lignes épipolaires
plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(img1_rect_epilines)
plt.subplot(122), plt.imshow(img2_rect_epilines)
plt.suptitle('Lignes épipolaires après rectification')
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Lecture des deux images
img1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png')
img2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png')

# Conversion en niveaux de gris
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Détection des points d'intérêt
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(gray1, None)
kp2, des2 = sift.detectAndCompute(gray2, None)

# Appariement des points d'intérêt
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)

# Extraction des points appariés
pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])

# Calcul de la matrice fondamentale
F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)

# Sélection des bons appariements
pts1 = pts1[mask.ravel() == 1]
pts2 = pts2[mask.ravel() == 1]

# Fonction pour dessiner les lignes épipolaires
def draw_epilines(img1, img2, lines, pts1, pts2):
    r, c = img1.shape[:2]
    img1_color = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2_color = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1_color = cv2.line(img1_color, (x0, y0), (x1, y1), color, 1)
        img1_color = cv2.circle(img1_color, tuple(map(int, pt1)), 5, color, -1)
        img2_color = cv2.circle(img2_color, tuple(map(int, pt2)), 5, color, -1)
    return img1_color, img2_color

# Calcul des lignes épipolaires
lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2, F)
lines1 = lines1.reshape(-1, 3)
img1_epilines, img2_epilines = draw_epilines(gray1, gray2, lines1, pts1, pts2)

# Combinaison des images avant rectification
combined_before = np.hstack((img1_epilines, img2_epilines))

# Rectification des images
h1, w1 = gray1.shape
h2, w2 = gray2.shape

_, H1, H2 = cv2.stereoRectifyUncalibrated(np.float32(pts1), np.float32(pts2), F, imgSize=(w1, h1))

img1_rectified = cv2.warpPerspective(img1, H1, (w1, h1))
img2_rectified = cv2.warpPerspective(img2, H2, (w2, h2))

gray1_rectified = cv2.warpPerspective(gray1, H1, (w1, h1))
gray2_rectified = cv2.warpPerspective(gray2, H2, (w2, h2))

# Recalcul des points appariés après rectification
kp1_rectified, des1_rectified = sift.detectAndCompute(gray1_rectified, None)
kp2_rectified, des2_rectified = sift.detectAndCompute(gray2_rectified, None)
matches_rectified = bf.match(des1_rectified, des2_rectified)
matches_rectified = sorted(matches_rectified, key=lambda x: x.distance)

pts1_rectified = np.float32([kp1_rectified[m.queryIdx].pt for m in matches_rectified])
pts2_rectified = np.float32([kp2_rectified[m.trainIdx].pt for m in matches_rectified])

# Affichage des lignes épipolaires après rectification
lines1_rectified = cv2.computeCorrespondEpilines(pts2_rectified.reshape(-1, 1, 2), 2, F)
lines1_rectified = lines1_rectified.reshape(-1, 3)
img1_rect_epilines, img2_rect_epilines = draw_epilines(gray1_rectified, gray2_rectified, lines1_rectified, pts1_rectified, pts2_rectified)

# Combinaison des images après rectification
combined_after = np.hstack((img1_rect_epilines, img2_rect_epilines))

# Affichage des résultats
plt.figure(figsize=(15, 5))
plt.subplot(121)
plt.imshow(combined_before, cmap='gray')
plt.title('Lignes épipolaires avant rectification')
plt.subplot(122)
plt.imshow(combined_after, cmap='gray')
plt.title('Lignes épipolaires après rectification')
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to find features in the two images
def feature_detection(img1, img2):
    sift = cv2.SIFT_create()
    kp1, desc1 = sift.detectAndCompute(img1, None)
    kp2, desc2 = sift.detectAndCompute(img2, None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc1, desc2, k=2)

    # Extracting good matches
    good = []
    for m, n in matches:
        if m.distance < 0.1 * n.distance:
            good.append(m)

    left_pts = np.float32([kp1[m.queryIdx].pt for m in good])
    right_pts = np.float32([kp2[m.trainIdx].pt for m in good])
    return left_pts, right_pts, kp1, kp2, good

# Function to compute Fundamental Matrix given feature points in two images
def fundamental_matrix(left_pts, right_pts):
    F, mask = cv2.findFundamentalMat(left_pts, right_pts, cv2.FM_RANSAC)
    return F, mask

# Function to draw epilines on the images
def draw_epilines(img1, img2, lines, pts1, pts2):
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 2)
        img1 = cv2.circle(img1, (int(pt1[0]), int(pt1[1])), 3, color, -1)
        img2 = cv2.circle(img2, (int(pt2[0]), int(pt2[1])), 3, color, -1)
    return img1, img2

# Load images
img1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png', cv2.IMREAD_GRAYSCALE)

# Detect features and compute fundamental matrix using RANSAC
left_pts, right_pts, kp1, kp2, good = feature_detection(img1, img2)
F, mask = fundamental_matrix(left_pts, right_pts)

# Select inlier points
left_pts = left_pts[mask.ravel() == 1]
right_pts = right_pts[mask.ravel() == 1]

# Compute epilines for both images
lines1 = cv2.computeCorrespondEpilines(right_pts.reshape(-1, 1, 2), 2, F)
lines1 = lines1.reshape(-1, 3)
img1_with_lines, img2_with_lines = draw_epilines(img1, img2, lines1, left_pts, right_pts)

lines2 = cv2.computeCorrespondEpilines(left_pts.reshape(-1, 1, 2), 1, F)
lines2 = lines2.reshape(-1, 3)
img2_with_lines, img1_with_lines = draw_epilines(img2, img1, lines2, right_pts, left_pts)

# Combine images with epilines side by side
img_combined_with_lines = np.hstack((img1_with_lines, img2_with_lines))

# Display combined image with epilines
plt.figure(figsize=(20, 10))
plt.imshow(img_combined_with_lines)
plt.title('Combined Image with Epipolar Lines (Without Rectification)')
plt.axis('off')
plt.savefig('/content/drive/MyDrive/COMPUTER_VISION/epilines_without_rectification.png')
plt.show()

# Stereo rectification
h1, w1 = img1.shape[:2]
_, H1, H2 = cv2.stereoRectifyUncalibrated(np.float32(left_pts), np.float32(right_pts), F, imgSize=(w1, h1))

# Rectify the images
img1_rectified = cv2.warpPerspective(img1, H1, (w1, h1))
img2_rectified = cv2.warpPerspective(img2, H2, (w1, h1))

# Combine rectified images side by side
img_combined_rectified = np.hstack((img1_rectified, img2_rectified))

# Compute epilines for the rectified images
left_pts_rect = cv2.perspectiveTransform(left_pts.reshape(-1, 1, 2), H1).reshape(-1, 2)
right_pts_rect = cv2.perspectiveTransform(right_pts.reshape(-1, 1, 2), H2).reshape(-1, 2)

lines1_rect = cv2.computeCorrespondEpilines(right_pts_rect.reshape(-1, 1, 2), 2, F)
lines1_rect = lines1_rect.reshape(-1, 3)
img1_rect_with_lines, img2_rect_with_lines = draw_epilines(img1_rectified, img2_rectified, lines1_rect, left_pts_rect, right_pts_rect)

lines2_rect = cv2.computeCorrespondEpilines(left_pts_rect.reshape(-1, 1, 2), 1, F)
lines2_rect = lines2_rect.reshape(-1, 3)
img2_rect_with_lines, img1_rect_with_lines = draw_epilines(img2_rectified, img1_rectified, lines2_rect, right_pts_rect, left_pts_rect)

# Combine rectified images with epilines side by side
img_combined_rect_with_lines = np.hstack((img1_rect_with_lines, img2_rect_with_lines))

# Display rectified combined images with epilines
plt.figure(figsize=(20, 10))
plt.imshow(img_combined_rect_with_lines)
plt.title('Rectified Combined Image with Epipolar Lines')
plt.axis('off')
plt.savefig('/content/drive/MyDrive/COMPUTER_VISION/epilines_with_rectification.png')
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to find features in the two images
def feature_detection(img1, img2):
    sift = cv2.SIFT_create()
    kp1, desc1 = sift.detectAndCompute(img1, None)
    kp2, desc2 = sift.detectAndCompute(img2, None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc1, desc2, k=2)

    # Extracting good matches
    good = []
    for m, n in matches:
        if m.distance < 0.1 * n.distance:
            good.append(m)

    left_pts = np.float32([kp1[m.queryIdx].pt for m in good])
    right_pts = np.float32([kp2[m.trainIdx].pt for m in good])
    return left_pts, right_pts, kp1, kp2, good

# Function to compute Fundamental Matrix given feature points in two images
def fundamental_matrix(left_pts, right_pts):
    F, mask = cv2.findFundamentalMat(left_pts, right_pts, cv2.FM_RANSAC)
    return F, mask

# Function to draw epilines on the images
def draw_epilines(img1, img2, lines, pts1, pts2):
    r, c = img1.shape
    img1_color = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2_color = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1_color = cv2.line(img1_color, (x0, y0), (x1, y1), color, 2)
        img2_color = cv2.line(img2_color, (x0, y0), (x1, y1), color, 2)
        img1_color = cv2.circle(img1_color, (int(pt1[0]), int(pt1[1])), 3, color, -1)
        img2_color = cv2.circle(img2_color, (int(pt2[0]), int(pt2[1])), 3, color, -1)
    return img1_color, img2_color

# Load images
img1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png', cv2.IMREAD_GRAYSCALE)

# Detect features and compute fundamental matrix using RANSAC
left_pts, right_pts, kp1, kp2, good = feature_detection(img1, img2)
F, mask = fundamental_matrix(left_pts, right_pts)

# Select inlier points
left_pts = left_pts[mask.ravel() == 1]
right_pts = right_pts[mask.ravel() == 1]

# Compute epilines for both images
lines1 = cv2.computeCorrespondEpilines(right_pts.reshape(-1, 1, 2), 2, F)
lines1 = lines1.reshape(-1, 3)
img1_with_lines, img2_with_lines = draw_epilines(img1, img2, lines1, left_pts, right_pts)

lines2 = cv2.computeCorrespondEpilines(left_pts.reshape(-1, 1, 2), 1, F)
lines2 = lines2.reshape(-1, 3)
img2_with_lines, img1_with_lines = draw_epilines(img2, img1, lines2, right_pts, left_pts)

# Combine images with epilines side by side
img_combined_with_lines = np.hstack((img1_with_lines, img2_with_lines))

# Display combined image with epilines
plt.figure(figsize=(20, 10))
plt.imshow(img_combined_with_lines)
plt.title('Combined Image with Epipolar Lines (Without Rectification)')
plt.axis('off')
plt.savefig('/content/drive/MyDrive/COMPUTER_VISION/epilines_without_rectification.png')
plt.show()

# Stereo rectification
h1, w1 = img1.shape[:2]
_, H1, H2 = cv2.stereoRectifyUncalibrated(np.float32(left_pts), np.float32(right_pts), F, imgSize=(w1, h1))

# Rectify the images
img1_rectified = cv2.warpPerspective(img1, H1, (w1, h1))
img2_rectified = cv2.warpPerspective(img2, H2, (w1, h1))

# Combine rectified images side by side
img_combined_rectified = np.hstack((img1_rectified, img2_rectified))

# Compute epilines for the rectified images
left_pts_rect = cv2.perspectiveTransform(left_pts.reshape(-1, 1, 2), H1).reshape(-1, 2)
right_pts_rect = cv2.perspectiveTransform(right_pts.reshape(-1, 1, 2), H2).reshape(-1, 2)

lines1_rect = cv2.computeCorrespondEpilines(right_pts_rect.reshape(-1, 1, 2), 2, F)
lines1_rect = lines1_rect.reshape(-1, 3)
img1_rect_with_lines, img2_rect_with_lines = draw_epilines(img1_rectified, img2_rectified, lines1_rect, left_pts_rect, right_pts_rect)

lines2_rect = cv2.computeCorrespondEpilines(left_pts_rect.reshape(-1, 1, 2), 1, F)
lines2_rect = lines2_rect.reshape(-1, 3)
img2_rect_with_lines, img1_rect_with_lines = draw_epilines(img2_rectified, img1_rectified, lines2_rect, right_pts_rect, left_pts_rect)

# Combine rectified images with epilines side by side
img_combined_rect_with_lines = np.hstack((img1_rect_with_lines, img2_rect_with_lines))

# Display rectified combined images with epilines
plt.figure(figsize=(20, 10))
plt.imshow(img_combined_rect_with_lines)
plt.title('Rectified Combined Image with Epipolar Lines')
plt.axis('off')
plt.savefig('/content/drive/MyDrive/COMPUTER_VISION/epilines_with_rectification.png')
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to find features in the two images
def feature_detection(img1, img2):
    sift = cv2.SIFT_create()
    kp1, desc1 = sift.detectAndCompute(img1, None)
    kp2, desc2 = sift.detectAndCompute(img2, None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc1, desc2, k=2)

    # Extracting good matches
    good = []
    for m, n in matches:
        if m.distance < 0.1 * n.distance:
            good.append(m)

    left_pts = np.float32([kp1[m.queryIdx].pt for m in good])
    right_pts = np.float32([kp2[m.trainIdx].pt for m in good])
    return left_pts, right_pts, kp1, kp2, good

# Function to compute Fundamental Matrix given feature points in two images
def fundamental_matrix(left_pts, right_pts):
    F, mask = cv2.findFundamentalMat(left_pts, right_pts, cv2.FM_RANSAC)
    return F, mask

# Function to draw epilines on the images
def draw_epilines(img1, img2, lines, pts1, pts2, colors):
    r, c = img1.shape
    img1_color = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2_color = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2, color in zip(lines, pts1, pts2, colors):
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1_color = cv2.line(img1_color, (x0, y0), (x1, y1), color, 2)
        img2_color = cv2.line(img2_color, (x0, y0), (x1, y1), color, 2)
        img1_color = cv2.circle(img1_color, (int(pt1[0]), int(pt1[1])), 3, color, -1)
        img2_color = cv2.circle(img2_color, (int(pt2[0]), int(pt2[1])), 3, color, -1)
    return img1_color, img2_color

# Load images
img1 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imLeft.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('/content/drive/MyDrive/COMPUTER_VISION/imRight.png', cv2.IMREAD_GRAYSCALE)

# Detect features and compute fundamental matrix using RANSAC
left_pts, right_pts, kp1, kp2, good = feature_detection(img1, img2)
F, mask = fundamental_matrix(left_pts, right_pts)

# Select inlier points
left_pts = left_pts[mask.ravel() == 1]
right_pts = right_pts[mask.ravel() == 1]

# Generate a fixed set of colors for correspondences
colors = [tuple(np.random.randint(0, 255, 3).tolist()) for _ in range(len(left_pts))]

# Compute epilines for both images
lines1 = cv2.computeCorrespondEpilines(right_pts.reshape(-1, 1, 2), 2, F)
lines1 = lines1.reshape(-1, 3)
img1_with_lines, img2_with_lines = draw_epilines(img1, img2, lines1, left_pts, right_pts, colors)

# Combine images with epilines side by side
img_combined_with_lines = np.hstack((img1_with_lines, img2_with_lines))

# Display combined image with epilines
plt.figure(figsize=(20, 10))
plt.imshow(img_combined_with_lines)
plt.title('Combined Image with Epipolar Lines (Without Rectification)')
plt.axis('off')
plt.show()

# Stereo rectification
h1, w1 = img1.shape[:2]
_, H1, H2 = cv2.stereoRectifyUncalibrated(np.float32(left_pts), np.float32(right_pts), F, imgSize=(w1, h1))

# Rectify the images
img1_rectified = cv2.warpPerspective(img1, H1, (w1, h1))
img2_rectified = cv2.warpPerspective(img2, H2, (w1, h1))

# Combine rectified images side by side
img_combined_rectified = np.hstack((img1_rectified, img2_rectified))

# Compute epilines for the rectified images
left_pts_rect = cv2.perspectiveTransform(left_pts.reshape(-1, 1, 2), H1).reshape(-1, 2)
right_pts_rect = cv2.perspectiveTransform(right_pts.reshape(-1, 1, 2), H2).reshape(-1, 2)

lines1_rect = cv2.computeCorrespondEpilines(right_pts_rect.reshape(-1, 1, 2), 2, F)
lines1_rect = lines1_rect.reshape(-1, 3)
img1_rect_with_lines, img2_rect_with_lines = draw_epilines(img1_rectified, img2_rectified, lines1_rect, left_pts_rect, right_pts_rect, colors)

# Combine rectified images with epilines side by side
img_combined_rect_with_lines = np.hstack((img1_rect_with_lines, img2_rect_with_lines))

# Display rectified combined images with epilines
plt.figure(figsize=(20, 10))
plt.imshow(img_combined_rect_with_lines)
plt.title('Rectified Combined Image with Epipolar Lines')
plt.axis('off')
plt.show()
